In [1]:
print('PyDev console: using IPython 7.5.0\n')


import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(['/home/dang/Documents/Doc/2021/mine'])

In [2]:


import time
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common import env_checker

from torch.utils.tensorboard import SummaryWriter
import minerl  # it's important to import minerl after SB3, otherwise model.save doesn't work...
import cv2
import numpy as np
import pickle
import torch
try:
    wandb = None
    import wandb
except ImportError:
    pass

class PovOnlyObservation(gym.ObservationWrapper):

    def __init__(self, env):
        super().__init__(env)
        # 64 x 64 + 64 x64 + 18 [cpa;, cobblestone]
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(8210,), dtype=np.float64)
        self.observation_space = self.env.observation_space['pov']
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(4114,), dtype=np.float64)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(64, 64, 2), dtype=np.uint8)

        self.names_stack = []
        self.max_name = 120
        self.max_item = 50

        dict_data = {}
        with open('data_1.pkl', 'rb') as f:
            dict_data = pickle.load(f)



        h_range = []
        s_range = []
        v_range = []

        names = []
        kind = []
        counter = 0
        tree = 0
        for k, v in dict_data.items():
            for i in range(len(v['H'])):
                h_range.append(v['H'][i])
                s_range.append(v['S'][i])
                v_range.append(v['V'][i])

                names.append(v['name'][i])
                if k == "tree_chunk":
                    kind.append(255)
                else:
                    kind.append(0)

            # if k == tre
            # counter += 1

        h_range = np.expand_dims(h_range, axis=0)
        h_range = np.repeat(h_range, 64, axis=0)
        h_range = np.expand_dims(h_range, axis=0)
        h_range = np.repeat(h_range, 64, axis=0)
        h_range = np.array(h_range, dtype=np.float64)
        self.h_range = torch.Tensor(h_range)

        s_range = np.expand_dims(s_range, axis=0)
        s_range = np.repeat(s_range, 64, axis=0)
        s_range = np.expand_dims(s_range, axis=0)
        s_range = np.repeat(s_range, 64, axis=0)
        s_range = np.array(s_range, dtype=np.float64)
        self.s_range = torch.Tensor(s_range)


        v_range = np.expand_dims(v_range, axis=0)
        v_range = np.repeat(v_range, 64, axis=0)
        v_range = np.expand_dims(v_range, axis=0)
        v_range = np.repeat(v_range, 64, axis=0)
        v_range = np.array(v_range, dtype=np.float64)
        self.v_range = torch.Tensor(v_range)

        self.kind = np.array(kind, dtype=np.uint8)

    def observation(self, observation):
        image = observation['pov']
        hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        hsv_64 = np.array(hsv, dtype=np.float64)
        hsv_64 = torch.Tensor(hsv_64)

        h = torch.unsqueeze(hsv_64[:, :, 0], -1)
        s = torch.unsqueeze(hsv_64[:, :, 1], -1)
        v = torch.unsqueeze(hsv_64[:, :, 2], -1)

        h_error = torch.abs(h - self.h_range)
        index = torch.where(h_error >= 89.5)
        h_error[index[0], index[1], index[2]] = 179 - h_error[index[0], index[1], index[2]]

        d_error = torch.sqrt(torch.pow(h_error, 2) + torch.pow((s / 4 - self.s_range / 4), 2)
                             + torch.pow((v / 6 - self.v_range / 6), 2))

        d_error = torch.argmin(d_error, axis=2)
        tree_obs = self.kind[d_error]
        tree_obs = np.array(tree_obs, dtype=np.uint8)

        hsv[:,:,0] = 100
        hsv[:,:,1] = 2

        obs = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
        obs = cv2.cvtColor(obs, cv2.COLOR_RGB2GRAY)

        obs = np.expand_dims(obs, axis=-1)

        tree_obs = np.expand_dims(tree_obs,axis=-1)
        obs = np.concatenate((obs,tree_obs), axis=-1)

        return obs

class ActionShaping(gym.ActionWrapper):

    def __init__(self, env, camera_angle=15, always_attack=False):
        super().__init__(env)

        self.camera_angle = camera_angle
        self.always_attack = False
        self._actions = [
            [('attack', 1)],
            [('forward', 1)],
            # [('back', 1)],
            # [('left', 1)],
            # [('right', 1)],
            # [('jump', 1)],
            # [('forward', 1), ('attack', 1)],
            # [('craft', 'planks')],
            [('forward', 1), ('jump', 1)],
            [('camera', [-self.camera_angle, 0])],
            [('camera', [self.camera_angle, 0])],
            [('camera', [0, self.camera_angle])],
            [('camera', [0, -self.camera_angle])],
        ]

        self.actions = []
        for actions in self._actions:
            act = self.env.action_space.noop()
            for a, v in actions:
                act[a] = v
            if self.always_attack:
                act['attack'] = 1
            self.actions.append(act)

        self.action_space = gym.spaces.Discrete(len(self.actions))

    def action(self, action):
        return self.actions[action]

def track_exp(project_name=None):
    config = {
        "TRAIN_TIMESTEPS": 1000000,  # number of steps to train the agent for. At 70 FPS 2m steps take about 8 hours.
        "TRAIN_ENV": 'MineRLTreechop-v0',
        # training environment for the RL agent. Could use MineRLObtainDiamondDense-v0 here.
        "TRAIN_MODEL_NAME": 'potato',  # name to use when saving the trained agent.
        "TEST_MODEL_NAME": 'potato',  # name to use when loading the trained agent.
        "TEST_EPISODES": 10,  # number of episodes to test the agent for.
        "MAX_TEST_EPISODE_LEN": 18000,  # 18k is the default for MineRLObtainDiamond.
        "TREECHOP_STEPS": 2000,  # number of steps to run RL lumberjack for in evaluations.
        "RECORD_TRAINING_VIDEOS": False,  # if True, records videos of all episodes done during training.
        "RECORD_TEST_VIDEOS": False,  # if True, records videos of all episodes done during evaluation.
    }
    wandb.init(
        anonymous="allow",
        project=project_name,
        config=config,
        sync_tensorboard=True,
        name='v1',
        monitor_gym=True,
        save_code=True,
    )


def make_env(idx):
    def thunk():
        env = gym.make('MineRLObtainDiamond-v0')
        if idx == 0 and False:
            env = gym.wrappers.Monitor(env, f"train_videos/{'v1'}")
        env = PovOnlyObservation(env)
        env = ActionShaping(env, always_attack=True)
        env = gym.wrappers.RecordEpisodeStatistics(env)  # record stats such as returns
        return env
    return thunk

track_exp(project_name="minerl")

env = DummyVecEnv([make_env(i) for i in range(1)])
# For all the PPO hyperparameters you could tune see this:
# https://github.com/DLR-RM/stable-baselines3/blob/6f822b9ed7d6e8f57e5a58059923a5b24e8db283/stable_baselines3/ppo/ppo.py#L16
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=f"runs/{'v1'}")
model.learn(total_timesteps=2000000)  # 2m steps is about 8h at 70 FPS
model.save('ppo_first')
# env_checker.check_env(env, warn=True, skip_render_check=True)
# MineRL might throw an exception when closing on Windows, but it can be ignored (the environment does close).
try:
    env.close()
except Exception:
    pass

# env.actions.sample()
# obs.dtype
# np.ndarray.dtype(obs)
# env = gym.make('MineRLObtainDiamond-v0')
# env = PovOnlyObservation(env)
#env = ActionShaping(env, always_attack=True)
# # obs = env.reset()
#
# # Parameters:
# config = {
#     "TRAIN_TIMESTEPS": 2000000,  # number of steps to train the agent for. At 70 FPS 2m steps take about 8 hours.
#     "TRAIN_ENV": 'MineRLTreechop-v0',  # training environment for the RL agent. Could use MineRLObtainDiamondDense-v0 here.
#     "TRAIN_MODEL_NAME": 'potato',  # name to use when saving the trained agent.
#     "TEST_MODEL_NAME": 'potato',  # name to use when loading the trained agent.
#     "TEST_EPISODES": 10,  # number of episodes to test the agent for.
#     "MAX_TEST_EPISODE_LEN": 18000,  # 18k is the default for MineRLObtainDiamond.
#     "TREECHOP_STEPS": 2000,  # number of steps to run RL lumberjack for in evaluations.
#     "RECORD_TRAINING_VIDEOS": False,  # if True, records videos of all episodes done during training.
#     "RECORD_TEST_VIDEOS": False,  # if True, records videos of all episodes done during evaluation.
# }
# experiment_name = f"ppo_{int(time.time())}"
#
#
#
#
#
#
#
#
# def train():
#     env = DummyVecEnv([make_env(i) for i in range(1)])
#     # For all the PPO hyperparameters you could tune see this:
#     # https://github.com/DLR-RM/stable-baselines3/blob/6f822b9ed7d6e8f57e5a58059923a5b24e8db283/stable_baselines3/ppo/ppo.py#L16
#     model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=f"runs/{experiment_name}")
#     model.learn(total_timesteps=config["TRAIN_TIMESTEPS"])  # 2m steps is about 8h at 70 FPS
#     model.save(config["TRAIN_MODEL_NAME"])
#
#     # MineRL might throw an exception when closing on Windows, but it can be ignored (the environment does close).
#     try:
#         env.close()
#     except Exception:
#         pass
#
#
# def str_to_act(env, actions):
#     """
#     Simplifies specifying actions for the scripted part of the agent.
#     Some examples for a string with a single action:
#         'craft:planks'
#         'camera:[10,0]'
#         'attack'
#         'jump'
#         ''
#     There should be no spaces in single actions, as we use spaces to separate actions with multiple "buttons" pressed:
#         'attack sprint forward'
#         'forward camera:[0,10]'
#
#     :param env: base MineRL environment.
#     :param actions: string of actions.
#     :return: dict action, compatible with the base MineRL environment.
#     """
#     act = env.action_space.noop()
#     for action in actions.split():
#         if ":" in action:
#             k, v = action.split(':')
#             if k == 'camera':
#                 act[k] = eval(v)
#             else:
#                 act[k] = v
#         else:
#             act[action] = 1
#     return act
#
#
# def get_action_sequence():
#     """
#     Specify the action sequence for the scripted part of the agent.
#     """
#     # make planks, sticks, crafting table and wooden pickaxe:
#     action_sequence = []
#     action_sequence += [''] * 100
#     action_sequence += ['craft:planks'] * 4
#     action_sequence += ['craft:stick'] * 2
#     action_sequence += ['craft:crafting_table']
#     action_sequence += ['camera:[10,0]'] * 18
#     action_sequence += ['attack'] * 20
#     action_sequence += [''] * 10
#     action_sequence += ['jump']
#     action_sequence += [''] * 5
#     action_sequence += ['place:crafting_table']
#     action_sequence += [''] * 10
#
#     # bug: looking straight down at a crafting table doesn't let you craft. So we look up a bit before crafting.
#     action_sequence += ['camera:[-1,0]']
#     action_sequence += ['nearbyCraft:wooden_pickaxe']
#     action_sequence += ['camera:[1,0]']
#     action_sequence += [''] * 10
#     action_sequence += ['equip:wooden_pickaxe']
#     action_sequence += [''] * 10
#
#     # dig down:
#     action_sequence += ['attack'] * 600
#     action_sequence += [''] * 10
#
#     return action_sequence
#
#
# def test():
#     action_sequence = get_action_sequence()
#     writer = SummaryWriter(f"runs/{experiment_name}")
#     env = gym.make('MineRLObtainDiamond-v0').env
#     time_limit = min(config["MAX_TEST_EPISODE_LEN"], config["TREECHOP_STEPS"] + len(action_sequence))
#     env = gym.wrappers.TimeLimit(env, time_limit)
#
#     # optional interactive mode, where you can connect to your agent and play together (see link for details):
#     # https://minerl.io/docs/tutorials/minerl_tools.html#interactive-mode-minerl-interactor
#     # env.make_interactive(port=6666, realtime=True)
#
#     if config["RECORD_TEST_VIDEOS"]:
#         env = gym.wrappers.Monitor(env, f"test_videos/{experiment_name}")
#     env = PovOnlyObservation(env)
#     env = ActionShaping(env, always_attack=True)
#     env1 = env.env
#
#     model = PPO.load(config["TEST_MODEL_NAME"], verbose=1)
#     model.set_env(env)
#
#     for episode in range(config["TEST_EPISODES"]):
#         obs = env.reset()
#         done = False
#         total_reward = 0
#         steps = 0
#
#         # RL part to get some logs:
#         for i in range(config["TREECHOP_STEPS"]):
#             action = model.predict(obs.copy())
#             obs, reward, done, _ = env.step(action[0])
#             total_reward += reward
#             steps += 1
#             if done:
#                 break
#
#         # scripted part to use the logs:
#         if not done:
#             for i, action in enumerate(action_sequence[:config["MAX_TEST_EPISODE_LEN"] - config["TREECHOP_STEPS"]]):
#                 obs, reward, done, _ = env1.step(str_to_act(env1, action))
#                 total_reward += reward
#                 steps += 1
#                 if done:
#                     break
#
#         print(f'Episode #{episode + 1} return: {total_reward}\t\t episode length: {steps}')
#         writer.add_scalar("return", total_reward, global_step=episode)
#
#     env.close()
#
#
# def main():
#     # uncomment the following to upload the logs and videos to Weights and Biases
#     # track_exp(project_name="minerl")
#
#     # train()
#     test()
#
#
# if __name__ == '__main__':
#     main()


env = gym.make('MineRLObtainDiamond-v0')
env = PovOnlyObservation(env)
env = ActionShaping(env, always_attack=True)
env1 = env.env


for episode in range(3):
    obs = env.reset()
    done = False
    total_reward = 0
    steps = 0

    # RL part to get some logs:
    for i in range(10000):
        action = model.predict(obs.copy())
        obs, reward, done, _ = env.step(action[0])
        total_reward += reward
        env.render()
        steps += 1
        if done:
            break

    # scripted part to use the logs:
    # if not done:
    #     for i, action in enumerate(action_sequence[:config["MAX_TEST_EPISODE_LEN"] - config["TREECHOP_STEPS"]]):
    #         obs, reward, done, _ = env1.step(str_to_act(env1, action))
    #         total_reward += reward
    #         steps += 1
    #         if done:
    #             break
    #
    # print(f'Episode #{episode + 1} return: {total_reward}\t\t episode length: {steps}')
    # writer.add_scalar("return", total_reward, global_step=episode)

env.close()

<IPython.core.display.HTML object>

In [3]:
env = gym.make('MineRLObtainDiamond-v0')
env = PovOnlyObservation(env)
env = ActionShaping(env, always_attack=True)
env1 = env.env


for episode in range(3):
    obs = env.reset()
    done = False
    total_reward = 0
    steps = 0

    # RL part to get some logs:
    for i in range(10000):
        action = model.predict(obs.copy())
        obs, reward, done, _ = env.step(action[0])
        total_reward += reward
        env.render()
        steps += 1
        if done:
            break

    # scripted part to use the logs:
    # if not done:
    #     for i, action in enumerate(action_sequence[:config["MAX_TEST_EPISODE_LEN"] - config["TREECHOP_STEPS"]]):
    #         obs, reward, done, _ = env1.step(str_to_act(env1, action))
    #         total_reward += reward
    #         steps += 1
    #         if done:
    #             break
    #
    # print(f'Episode #{episode + 1} return: {total_reward}\t\t episode length: {steps}')
    # writer.add_scalar("return", total_reward, global_step=episode)

env.close()

In [4]:
action

(0, None)

In [5]:
action[0]

0

In [6]:
obs = env.reset()

In [7]:


for episode in range(3):
    obs = env.reset()
    done = False
    total_reward = 0
    steps = 0

    # RL part to get some logs:
    for i in range(10000):
        action = model.predict(obs.copy())
        print(action)
        obs, reward, done, _ = env.step(action[0])
        total_reward += reward
        env.render()
        steps += 1
        if done:
            break

    # scripted part to use the logs:
    # if not done:
    #     for i, action in enumerate(action_sequence[:config["MAX_TEST_EPISODE_LEN"] - config["TREECHOP_STEPS"]]):
    #         obs, reward, done, _ = env1.step(str_to_act(env1, action))
    #         total_reward += reward
    #         steps += 1
    #         if done:
    #             break
    #
    # print(f'Episode #{episode + 1} return: {total_reward}\t\t episode length: {steps}')
    # writer.add_scalar("return", total_reward, global_step=episode)

In [8]:
obs, reward, done, _ = env.step(3)
total_reward += reward
env.render()
steps += 1

In [9]:
obs, reward, done, _ = env.step(3)
total_reward += reward
env.render()
steps += 1

In [10]:
obs, reward, done, _ = env.step(3)
total_reward += reward
env.render()
steps += 1

In [11]:
obs, reward, done, _ = env.step(3)
total_reward += reward
env.render()
steps += 1

In [12]:
obs, reward, done, _ = env.step(3)
total_reward += reward
env.render()
steps += 1

In [13]:
obs, reward, done, _ = env.step(3)
total_reward += reward
env.render()
steps += 1

In [14]:
for i in range(10000):
    #action = model.predict(obs.copy())
    print(action)
    obs, reward, done, _ = env.step(3)
    total_reward += reward
    env.render()
    steps += 1

In [15]:
#action = model.predict(obs.copy())
print(action)
obs, reward, done, _ = env.step(2)
total_reward += reward
env.render()
steps += 1

In [16]:
for i in range(10000):
    #action = model.predict(obs.copy())
    print(action)
    obs, reward, done, _ = env.step(2)
    total_reward += reward
    env.render()
    steps += 1

In [17]:
print(action)
obs, reward, done, _ = env.step(4)
total_reward += reward
env.render()
steps += 1

In [18]:
obs = env.reset()

In [19]:
for i in range(10000):
    #action = model.predict(obs.copy())
    print(action)
    obs, reward, done, _ = env.step(4)
    total_reward += reward
    env.render()
    steps += 1

In [20]:
import minerl
import gym
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np, cv2
from pynput.keyboard import Key, Listener
from pynput.mouse import Button
from pynput import mouse
from threading import Thread
import pickle
import time
import random
import imutils
from pairing import pair, depair
import torch
torch.cuda.set_device(0)
# class PixelConnect():
#     def __init__(self):
#         super.__init__()
#
# class CompareNeighbor():
#     def __init__(self, max_neighbor_pixel=1, image_size=64):
#         super.__init__()
#         x = np.linspace(-max_neighbor_pixel, max_neighbor_pixel, max_neighbor_pixel * 2 + 1)
#         self.neighbor_pixels =  [[], []]
#         for i in range(len(x)):
#             for j in range(len(y)):
#                 if x[i] != 0 and x[j] != 0:
#                     self.neighbor_pixels[0].append(x[i])
#                     self.neighbor_pixels[1].append(x[j])
#
#
#         x = np.arange(1, image_size, 2)
#         self.pixels_map = {}
#         for i in range(len(x)):
#             for j in range(len(x)):
#                 self.pixels_map[tuple((x[i], x[j]))] = PixelConnect()
#
#     def make_pixel_connect(self, image):
#         hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
#         for pixel in self.pixels_map:
#

a = 3

# obs = env.reset()


# env = gym.make('MineRLObtainDiamond-v0')
# obs = env.reset()
# record_p = []

# hsv_bond = {}
#
# key_list.stop()
# mouse_list.stop()




def nothing(x):
    pass


def trafer_hsv(dict):
    cv2.setTrackbarPos('HMin', 'image', dict['hMin'])
    cv2.setTrackbarPos('SMin', 'image', dict['sMin'])
    if 'vMin' in dict:
        cv2.setTrackbarPos('VMin', 'image', dict['vMin'])
    else:
        cv2.setTrackbarPos('VMin', 'image', 0)

    cv2.setTrackbarPos('HMax', 'image', dict['hMax'])
    cv2.setTrackbarPos('SMax', 'image', dict['sMax'])

    if 'vMax' in dict:
        cv2.setTrackbarPos('VMax', 'image', dict['vMax'])
    else:
        cv2.setTrackbarPos('VMax', 'image', 255)


def on_move(x, y):
    global attack_b


def on_click(x, y, button, pressed):
    global attack_b
    global action
    # print('{0} at {1}'.format(
    #     button if pressed else 'Released',
    #     (x, y)))
    if pressed:
        try:
            if button == Button.left:
                action['attack'] = 1
        except AttributeError:
            pass
    else:
        action['attack'] = 0
    global key_on_release
    if key_on_release == 'esc':
        return False


def on_scroll(x, y, dx, dy):
    global attack_b


def on_press(key):
    global key_on_press
    global action
    try:
        key_on_press = key.char
        if key_on_press == 'w' and action['back'] != 1:
            action['forward'] = 1

        if key_on_press == 's' and action['forward'] != 1:
            action['back'] = 1

        if key_on_press == 'a' and action['right'] != 1:
            action['left'] = 1
        if key_on_press == 'd' and action['left'] != 1:
            action['right'] = 1

        if key_on_press == 'q' and action['camera'][1] == 0:
            action['camera'][1] = -15

        if key_on_press == 'e' and action['camera'][1] == 0:
            action['camera'][1] = 15

        if key_on_press == 'r' and action['camera'][0] == 0:
            action['camera'][0] = -15
        if key_on_press == 'f' and action['camera'][0] == 0:
            action['camera'][0] = 15


    except AttributeError:
        # do something when a certain key is pressed, using key, not key.char
        pass
    if key == Key.space:
        action['jump'] = 1


def on_release(key):
    global key_on_release
    global jump
    global action
    global pasue
    global lock
    try:
        key_on_release = key.char
        if key_on_release == 'w':
            action['forward'] = 0
        if key_on_release == 's':
            action['back'] = 0
        if key_on_release == 'a':
            action['left'] = 0
        if key_on_release == 'd':
            action['right'] = 0

        if key_on_release == 'q':
            action['camera'][1] = 0
        if key_on_release == 'e':
            action['camera'][1] = 0
        if key_on_release == 'r':
            action['camera'][0] = 0
        if key_on_release == 'f':
            action['camera'][0] = 0
        if key_on_release == 'p':
            pasue = not pasue
            print("pause ", pasue)
        if key_on_release == 'l':
            lock = not lock
            print(lock)
        # if key.char == 'o':
        #     return False
    except AttributeError:
        # do something when a certain key is pressed, using key, not key.char
        pass

    if key == Key.esc:
        # Stop listener
        print("Stop lister ")
        key_on_release = 'esc'
        return False
    if key == Key.space:
        action['jump'] = 0

def shift_image(X, dx, dy):
    X = np.array(X, dtype=np.float64)
    X = np.roll(X, dy, axis=0)
    X = np.roll(X, dx, axis=1)
    if dy>0:
        X[:dy, :] = 1500
    elif dy<0:
        X[dy:, :] = 1500
    if dx>0:
        X[:, :dx] = 1500
    elif dx<0:
        X[:, dx:] = 1500
    return X

def Life2CodingRGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  # checks mouse moves
        global hsv
        #image_2 = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        colorsBGR = hsv[y, x]
        # colorsRGB=cv2.cvtColor(colorsBGR,cv2.COLOR_BGR2HSV)
        global tmp_33
        global names
        global dict_data
        global h_range
        global s_range
        global v_range
        global X_VALUE
        global Y_VALUE
        X_VALUE = y
        Y_VALUE = x
        H_VALUE = colorsBGR[0]
        S_VALUE = colorsBGR[1]
        V_VALUE = colorsBGR[2]
        index = tmp_33[y,x]

        print("RGB Value at ({},{}):{} {} {}".format(x,y,colorsBGR, names[index],
                                                     [h_range[y,x,index],
                                                      s_range[y,x,index],
                                                      v_range[y,x,index]]), end='\r')

def Life2CodingRGB2(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  # checks mouse moves
        image_2  =cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        colorsBGR = image_2[y, x]
        # colorsRGB=cv2.cvtColor(colorsBGR,cv2.COLOR_BGR2HSV)
        global tmp_33
        global names
        print("RGB Value at ({},{}):{} {}".format(x,y,colorsBGR, names[tmp_33[y,x]]), end='\r')


cv2.namedWindow('mask', cv2.WINDOW_KEEPRATIO)
cv2.resizeWindow('mask', 1000, 1000)
cv2.moveWindow('mask', 1100, 30)
cv2.namedWindow('image', cv2.WINDOW_KEEPRATIO)
cv2.resizeWindow('image', 1000, 1000)
cv2.moveWindow('image', -30, 30)



cv2.namedWindow('color_range')
cv2.moveWindow('color_range', 40, 1000)

# cv2.createTrackbar('HMin', 'image', 0, 179, nothing)
# cv2.createTrackbar('SMin', 'image', 0, 255, nothing)
# cv2.createTrackbar('HMax', 'image', 0, 179, nothing)
# cv2.createTrackbar('SMax', 'image', 0, 255, nothing)
# cv2.createTrackbar('VMin', 'image', 0, 255, nothing)
# cv2.createTrackbar('VMax', 'image', 0, 255, nothing)
# # Set default value for Max HSV trackbars
# cv2.setTrackbarPos('HMax', 'image', 179)
# cv2.setTrackbarPos('SMax', 'image', 255)
# cv2.setTrackbarPos('VMax', 'image', 255)

# Initialize HSV min/max values
hMin = sMin = vMin = hMax = sMax = vMax = 0
phMin = psMin = pvMin = phMax = psMax = pvMax = 0

index = 1
action = env.action_space.noop()
key_list = Listener(on_press=on_press, on_release=on_release)
mouse_list = mouse.Listener(on_move=on_move, on_click=on_click, on_scroll=on_scroll)
attack_b = False
key_list.start()
mouse_list.start()

key_on_press = ''
key_on_release = ''
pasue = False
lock = True
levels = [-1, -1, -1]
current_level = -1
indexs = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

pool_size = 20
pool_colors = np.zeros([pool_size,3])
np.random.seed(10)
x = list(np.arange(0, 179, 179/pool_size))
pool_colors[:,0] =np.random.permutation(x)
pool_colors[:,1] = np.random.choice(range(50, 200), size=pool_size)
pool_colors[:,2] = np.random.choice(range(100, 200), size=pool_size)

names_stack = []
t_R = 0
font = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (400, 400)
fontScale = 0.6
fontColor = (0, 0, 255)
lineType = 1

max_size = 128 * 15

x = np.linspace(-1, 1, 3, dtype=np.int64)
# x = [-2, -1, 1, 2]
# neighbor_pixels = [[-1, 0, 1, 0, -2, 2, 2, -2], [0, 1, 0, -1, -2, -2, 2, 2]]
neighbor_pixels = [[], []]
for i in range(len(x)):
    for j in range(len(x)):
        if not (x[i] == 0 and x[j] == 0):
            neighbor_pixels[0].append(x[i])
            neighbor_pixels[1].append(x[j])
neighbor_pixels = np.array(neighbor_pixels)
H_VALUE = 0
S_VALUE = 0
V_VALUE = 0
X_VALUE = 0
Y_VALUE = 0
dict_data = {}
with open('data_1.pkl', 'rb') as f:
    dict_data = pickle.load(f)

h_range = []
s_range = []
v_range = []

names = []
kind = []
tree_chunk = []
for k, v in dict_data.items():
    for i in range(len(v['H'])):
        h_range.append(v['H'][i])
        s_range.append(v['S'][i])
        v_range.append(v['V'][i])

        names.append(v['name'][i])
        kind.append(k)
        if k == "tree_chunk":
            tree_chunk.append(255)
        else:
            tree_chunk.append(0)

x = np.sort(h_range)
h_range = np.expand_dims(h_range, axis=0)
h_range = np.repeat(h_range, 64, axis=0)
h_range = np.expand_dims(h_range, axis=0)
h_range = np.repeat(h_range, 64, axis=0)
h_range = np.array(h_range, dtype=np.float64)
h_range = torch.Tensor(h_range)

s_range = np.expand_dims(s_range, axis=0)
s_range = np.repeat(s_range, 64, axis=0)
s_range = np.expand_dims(s_range, axis=0)
s_range = np.repeat(s_range, 64, axis=0)
s_range = np.array(s_range, dtype=np.float64)
s_range = torch.Tensor(s_range)

v_range = np.expand_dims(v_range, axis=0)
v_range = np.repeat(v_range, 64, axis=0)
v_range = np.expand_dims(v_range, axis=0)
v_range = np.repeat(v_range, 64, axis=0)
v_range = np.array(v_range, dtype=np.float64)
v_range = torch.Tensor(v_range)



tree_chunk = np.array(tree_chunk, dtype=np.uint8)

keyboard_map = {'c': 'tree_chunk', 't': 'tree_leave', 'r': 'rock', 's': 'sand', 'd': 'dirt',
                'u': 'not_common', 'g':'grass','b':'black','w': 'water', 'k': 'sky', 'n':'snow',
                'm':'monster', 'a':'animal', 'h': 'coal', 'i': 'iron'}
key_selected = ''
add_done = False
while (True):
    # Display result image
    color_range = np.zeros([128, max_size, 3], dtype=np.uint8)
    tmp_time = time.time()

    if pasue:
        # action['place'] = 'furnace'
        # action['place'] = 'torch'

        # action['nearbyCraft'] = 'iron_pickaxe'
        # action['nearbyCraft'] = 'stone_pickaxe'

        obs, reward, done, _ = env.step(action)
        t_R += reward
        # obs['inventory']
        # print(obs['inventory']['iron_ore'], end='\r')

        # record_p.append(obs['pov'])
    else:
        action = env.action_space.noop()
    # if action['jump'] == 1:
    #     action['jump'] = 0
    image = obs['pov']

    # Convert to HSV format and color threshold
    X = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)


    tmp_mask = np.zeros([64, 64, 3], dtype=np.uint8)

    hsv_64 = np.array(hsv, dtype=np.float64)
    hsv_64 = torch.Tensor(hsv_64)
    pixels_maps = {}
    keys_mask = {}
    key_index = 0

    H_nei = np.zeros((64, 64, len(neighbor_pixels[0])), dtype=np.float64)
    S_nei = np.zeros((64, 64, len(neighbor_pixels[0])), dtype=np.float64)
    V_nei = np.zeros((64, 64, len(neighbor_pixels[0])), dtype=np.float64)

    for counter in range(len(neighbor_pixels[0])):
        new_hsv = shift_image(hsv_64, neighbor_pixels[0][counter],
                              neighbor_pixels[1][counter])
        # zero_index = np.where(new_hsv[:, :, 0] == 1500)
        # new_hsv[zero_index[0], zero_index[1], :] =
        new_hsv = np.abs(hsv_64 - new_hsv)
        index = np.where(new_hsv[:,:,0] >= 89.5)
        new_hsv[index[0], index[1], 0] = np.abs(179 - new_hsv[index[0], index[1], 0])
        H_nei[:, :, counter] = new_hsv[:, :, 0]
        S_nei[:, :, counter] = new_hsv[:, :, 1]
        V_nei[:, :, counter] = new_hsv[:, :, 1]


    # for i in range(64):
    #     for j in range(64):
    #         pixel_id = tuple((i, j))
    #         pixel_hsv = hsv_64[i, j, :]
    #         H_error = H_nei[i, j]
    #         H_error_min = np.min(H_error)
    #         S_error = S_nei[i, j]
    #         V_error = V_nei[i, j]
    #
    #         if H_error_min <= 1:
    #             key = None
    #             for ins in range(len(H_error)):
    #                 next_pixel = tuple((i - neighbor_pixels[1][ins],
    #                                     j - neighbor_pixels[0][ins]))
    #
    #                 if H_error[ins] <= 1 and S_error[ins] <= 1 and V_error[ins] <= 1:
    #                     # if next_pixel[0] < 0:
    #                     #     print(i, j, neighbor_pixels[0][ins])
    #                     if key is None:
    #                         if pixel_id in pixels_maps:
    #                             key = pixels_maps[pixel_id]
    #
    #                         elif next_pixel in pixels_maps:
    #                             key = pixels_maps[next_pixel]
    #                         else:
    #                             key = key_index
    #                             keys_mask[key] = [[], [], {}, [], []]
    #                             key_index += 1
    #                     # print(key)
    #                     pixels_maps[pixel_id] = key
    #                     pixels_maps[next_pixel] = key
    #                     if pixel_id not in keys_mask[key][2]:
    #                         keys_mask[key][2][pixel_id] = 0
    #                         keys_mask[key][0].append(pixel_id[0])
    #                         keys_mask[key][1].append(pixel_id[1])
    #
    #                     if next_pixel not in keys_mask[key][2]:
    #                         keys_mask[key][2][next_pixel] = 0
    #                         keys_mask[key][0].append(next_pixel[0])
    #                         keys_mask[key][1].append(next_pixel[1])

    # # print(len(keys_mask))
    # max = 0
    # k_m = 0
    # for k, v in keys_mask.items():
    #     hsv_64[v[0], v[1], 0] = np.mean(hsv[v[0], v[1], 0])
    #     hsv_64[v[0], v[1], 1] = np.mean(hsv[v[0], v[1], 1])
    #     hsv_64[v[0], v[1], 2] = np.mean(hsv[v[0], v[1], 2])
    #     # hsv[v[3], v[4], 0] = np.mean(hsv[v[3], v[4], 0])
    #     # hsv[v[3], v[4], 1] = np.mean(hsv[v[3], v[4], 1])
    #     # tmp_mask[v[3], v[4], 2] = np.mean(hsv[v[3], v[4], 2]) + 20
    # len(keys_mask)
    tmp_time = time.time()


    h = np.expand_dims(hsv_64[:, :, 0], axis=-1)
    s = np.expand_dims(hsv_64[:, :, 1], axis=-1)
    v = np.expand_dims(hsv_64[:, :, 2], axis=-1)
    h = torch.unsqueeze(hsv_64[:,:,0], -1)
    s = torch.unsqueeze(hsv_64[:,:,1], -1)
    v = torch.unsqueeze(hsv_64[:,:,2], -1)
    #
    # h = torch.Tensor(h)
    # s = torch.Tensor(s)
    # v = torch.Tensor(v)
    v.shape
    # H1_range.shape

    if len(dict_data['rock']['H']) != 0:

        h_error = torch.abs(h - h_range)
        index = torch.where(h_error >= 89.5)
        h_error[index[0], index[1], index[2]] = 179 - h_error[index[0], index[1], index[2]]


        tmp_33 = torch.sqrt(torch.pow(h_error, 2) + torch.pow((s/4 - s_range/4), 2)
                            + torch.pow((v/6 - v_range/6), 2))

        tmp_33 = torch.argmin(tmp_33, axis=2)
        # print(time.time() - tmp_time)
        x = tree_chunk[tmp_33]
        # tmp_image = np.expand_dims(tmp_image,axis=-1)
        x = np.expand_dims(x,axis=-1)
        x.shape
        index_color = 0
        for i in range(64):
            for j in range(64):
                pixel = hsv_64[i, j, :]
                index_x = tmp_33[i,j]
                kins = kind[index_x]
                # if kins not in kindss:
                #     kindss[kins] = index_color
                #     index_color += 1

                # tmp_mask[i, j, :] = np.array([hsv_r[names[index_x]]['H'],
                #                               hsv_r[names[index_x]]['S'],
                #                               hsv_r[names[index_x]]['v']], dtype=np.uint8)

                if kins in names_stack:
                    index_1 = np.where(np.array(names_stack) == kins)[0][0]
                else:
                    names_stack.append(kins)
                    index_1 = len(names_stack)
                color = pool_colors[index_1]
                tmp_mask[i, j, :] = color

                if (index_1 + 1) * 128 < max_size:
                    if color_range[0, index_1 * 128 +10, 0] == 0:
                        color_range[:, index_1 * 128: (index_1 + 1) * 128, :] = color
                        cv2.putText(color_range, kins,
                                    (index_1 * 128 + 20, 64),
                                    font,
                                    fontScale,
                                    fontColor,
                                    lineType)
        # print(time.time() - tmp_time)

    # tmp_33[29,22]
    if not lock and not pasue:
        a = 3
        if key_on_press in keyboard_map and key_on_release != 'l':
            key_selected = keyboard_map[key_on_press]
            print("new: key {0} - {1} - {2}".format(key_selected, [hsv_64[X_VALUE,Y_VALUE, 0],
                                                                   hsv_64[X_VALUE,Y_VALUE, 1],
                                                                   hsv_64[X_VALUE,Y_VALUE, 2]], [X_VALUE, Y_VALUE]))
            key_on_press = ''
            add_done = False

        if key_selected != '':
            if key_on_press in indexs:
                if int(key_on_press) == 1:
                    dict_data[key_selected]['H'].append(hsv_64[X_VALUE,Y_VALUE,0])
                    dict_data[key_selected]['S'].append(hsv_64[X_VALUE,Y_VALUE,1])
                    dict_data[key_selected]['V'].append(hsv_64[X_VALUE,Y_VALUE,2])

                    name = '{0}_{1}'.format(key_selected, len(dict_data[key_selected]['H']))

                    dict_data[key_selected]['name'].append(name)

                    print("ADDED : key {0} - {1} - {2}".format(key_selected, [H_VALUE, S_VALUE, V_VALUE],
                                                               [X_VALUE, Y_VALUE]))
                    with open('data_1.pkl', 'wb') as f:
                        pickle.dump(dict_data, f)


                if int(key_on_press) == 2:
                    print("STOP")
                    key_selected = ''
                    add_done = False
                if int(key_on_press) == 4:
                    h_del = dict_data[key_selected]['H'][-1]
                    s_del = dict_data[key_selected]['S'][-1]
                    v_del = dict_data[key_selected]['V'][-1]
                    name_del = dict_data[key_selected]['name'][-1]
                    dict_data[key_selected]['H'] = dict_data[key_selected]['H'][:-1]
                    dict_data[key_selected]['S'] = dict_data[key_selected]['S'][:-1]
                    dict_data[key_selected]['V'] = dict_data[key_selected]['V'][:-1]
                    dict_data[key_selected]['name'] = dict_data[key_selected]['name'][:-1]


                    print("DEL : key {0} - {1}".format(key_selected, [h_del, s_del, v_del]))
                    # key_selected = ''
                    add_done = False
                    with open('data_1.pkl', 'wb') as f:
                        pickle.dump(dict_data, f)

                h_range = []
                s_range = []
                v_range = []

                names = []
                kind = []
                for k, v in dict_data.items():
                    for i in range(len(v['H'])):
                        h_range.append(v['H'][i])
                        s_range.append(v['S'][i])
                        v_range.append(v['V'][i])

                        names.append(v['name'][i])
                        kind.append(k)
                print(len(h_range))

                h_range = np.expand_dims(h_range, axis=0)
                h_range = np.repeat(h_range, 64, axis=0)
                h_range = np.expand_dims(h_range, axis=0)
                h_range = np.repeat(h_range, 64, axis=0)
                h_range = np.array(h_range, dtype=np.float64)
                h_range = torch.Tensor(h_range)

                s_range = np.expand_dims(s_range, axis=0)
                s_range = np.repeat(s_range, 64, axis=0)
                s_range = np.expand_dims(s_range, axis=0)
                s_range = np.repeat(s_range, 64, axis=0)
                s_range = np.array(s_range, dtype=np.float64)
                s_range = torch.Tensor(s_range)

                v_range = np.expand_dims(v_range, axis=0)
                v_range = np.repeat(v_range, 64, axis=0)
                v_range = np.expand_dims(v_range, axis=0)
                v_range = np.repeat(v_range, 64, axis=0)
                v_range = np.array(v_range, dtype=np.float64)
                v_range = torch.Tensor(v_range)

                add_done = True
                key_on_press = ''
        #image = cv2.imread("./block_image/Mega_Spruce_Tree.png")

    # print(np.count_nonzero(names == 32)/4096, end='\r')
    # mask = 255 - mask
    # hsv[:,:,2] = 255
    # X = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)



    # X = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

    # dst = cv2.addWeighted(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), 0.7,
    #                       cv2.cvtColor(tmp_mask, cv2.COLOR_HSV2RGB), 0.7, 0)
    #hsv = np.array(hsv_64,dtype=np.uint8)
    # hsv[48,19,0] = hsv[48,18,0]
    # hsv[48,19,1] = hsv[48,18,1]
    #hsv[48,19,2]=  hsv[48,18,2]

    tmp_hsv = np.array(hsv)
    #tmp_hsv[X_VALUE,Y_VALUE,0] = 50

    tmp_hsv[X_VALUE,Y_VALUE,1] = 255

    tmp_hsv[X_VALUE,Y_VALUE,2] = 230
    #tmp_hsv[:,:,2] = 255

    # index = np.where((hsv[:, :, 0] >=20) |( hsv[:, :, 0] <= 16))
    #hsv[index[0], index[1], :] = [0,0, 255]
    # tmp_hsv[:,:,0] = 179
    # tmp_hsv[:,:,1] = 2
    # tmp_hsv[:,:,2] = tmp_hsv[:,:,2] + 50

    tmp_image = cv2.cvtColor(tmp_hsv, cv2.COLOR_HSV2BGR)
    # tmp_image = cv2.cvtColor(tmp_image, cv2.COLOR_BGR2GRAY)

    # tmp_image[0,1]
    # tmp_image[X_VALUE,Y_VALUE,:] = [100,100,100]
    tmp_mask[X_VALUE,Y_VALUE,2] = 230

    cv2.imshow('image',tmp_image)
    cv2.resizeWindow('image', 950, 950)
    cv2.setMouseCallback('image', Life2CodingRGB)

    cv2.imshow('mask', cv2.cvtColor(tmp_mask, cv2.COLOR_HSV2RGB))
    cv2.resizeWindow('mask', 950, 950)
    cv2.setMouseCallback('mask', Life2CodingRGB)


    cv2.imshow('color_range', cv2.cvtColor(color_range, cv2.COLOR_HSV2RGB))
    # cv2.resizeWindow('mask', 800, 800)

    # for c in cnts:
    #     x, y, w, h = cv2.boundingRect(c)
    #
    #     print(cv2.contourArea(c))
    # print("---")
    # image = obs['pov']
    # cv2.imshow('image', cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # print(time.time() - tmp_time)

    if cv2.waitKey(10) & 0xFF == ord('o'):
        break
    # time.sleep(0.1)

key_list.stop()
mouse_list.stop()

cv2.destroyAllWindows()
hsv[16,60]
# with open('data_1.pkl', 'wb') as f:
#     pickle.dump(dict_data, f)


# pixels_maps = {}
# keys_mask = {}
# key_index = 0
# hsv_64 = np.array(hsv, dtype=np.float64)
# for i in range(64):
#     for j in range(64):
#
#         pixel_id = tuple((i, j))
#         pixel_hsv = hsv_64[i, j, :]
#
#         neigh_bor = [neighbor_pixels[0] + i, neighbor_pixels[1] + j]
#         index = np.where(((neigh_bor[0] <= 63) & (neigh_bor[0] > 0)))
#         neigh_bor[0] = neigh_bor[0][index]
#         neigh_bor[1] = neigh_bor[1][index]
#         index = np.where(((neigh_bor[1] <= 63) & (neigh_bor[1] > 0)))
#         neigh_bor[0] = neigh_bor[0][index]
#         neigh_bor[1] = neigh_bor[1][index]
#
#         neigh_hs = hsv_64[neigh_bor[0], neigh_bor[1], : 2]
#
#         HS_error = np.sum(np.abs(neigh_hs - pixel_hsv[:2]), axis=1) / 2
#         HS_error_min = np.min(HS_error)
#         if HS_error_min <= 25:
#             for ins in range(len(HS_error)):
#                 if np.abs(HS_error[ins] - HS_error_min) <= 5:
#                     next_pixel = tuple((neigh_bor[0][ins], neigh_bor[1][ins]))
#                     if next_pixel in pixels_maps:
#                         key = pixels_maps[next_pixel]
#                     else:
#                         key = key_index
#                         keys_mask[key] = [[], [], {}]
#                         key_index += 1
#                     pixels_maps[pixel_id] = key
#                     pixels_maps[next_pixel] = key
#                     if pixel_id not in keys_mask[key][2]:
#                         keys_mask[key][2][pixel_id] = 0
#                         keys_mask[key][0].append(pixel_id[0])
#                         keys_mask[key][1].append(pixel_id[1])
#                     if next_pixel not in keys_mask[key][2]:
#                         keys_mask[key][2][next_pixel] = 0
#                         keys_mask[key][0].append(next_pixel[0])
#                         keys_mask[key][1].append(next_pixel[1])
# keys_mask[0]
# for k,v in keys_mask.items():
#     print(len(v[0]))
#     # tmp_mask[v[0], v[1],0] = np.mean(hsv[v[0], v[1],0])
#     # tmp_mask[v[0], v[1],1] = np.mean(hsv[v[0], v[1],1])
#     # tmp_mask[v[0], v[1],2] = np.mean(hsv[v[0], v[1],2])
a = 3
a = 3
# dict_data['dirt']
# dict_data['tree_chunk']

# dict_data = {}
# for k,v in keyboard_map.items():
#     kind = v
#     if kind not in dict_data:
#         dict_data[kind] = {'H': [], 'S': [], 'V': [],
#                            'H_1': [], 'S_1': [],
#                            'H_2': [], 'S_2': [],
#                            'name': []}
# with open('data_1.pkl', 'wb') as f:
#     pickle.dump(dict_data, f)
# dict_data[kind]['H'].append(v['H'])
# dict_data[kind]['S'].append(v['S'])
# dict_data[kind]['V'].append(v['v'])
# name = '{0}_{1}'.format(kind, len(dict_data[kind]['H']))
# dict_data[kind]['name'].append(name)
#


# with open('data.pkl', 'rb') as f:
#     output = pickle.load(f)
#
# h_range = []
# s_range = []
# v_range = []
# names = []
# kind = []
# for k,v in output.items():
#     for i in range(len(v['H'])):
#         h_range.append(v['H'][i])
#         s_range.append(v['S'][i])
#         v_range.append(v['V'][i])
#         names.append(v['name'][i])
#         kind.append(k)
# h_range = np.expand_dims(h_range,axis=0)
# h_range = np.repeat(h_range, 64, axis=0)
# h_range = np.expand_dims(h_range,axis=0)
# h_range = np.repeat(h_range, 64, axis=0)
#
# s_range = np.expand_dims(s_range,axis=0)
# s_range = np.repeat(s_range, 64, axis=0)
# s_range = np.expand_dims(s_range,axis=0)
# s_range = np.repeat(s_range, 64, axis=0)
# h_range = np.array(h_range, dtype=np.float64)
# s_range = np.array(s_range, dtype=np.float64)
#
# v_range = np.expand_dims(v_range,axis=0)
# v_range = np.repeat(v_range, 64, axis=0)
# v_range = np.expand_dims(v_range,axis=0)
# v_range = np.repeat(v_range, 64, axis=0)
# v_range = np.array(v_range, dtype=np.float64)

# key_del = 'tree_chunk'
# item_del = 'tree_chunk_4'
# tmp_key = {'H': [], 'S': [], 'V': [],
#                            'D': [],
#                            'name': []}
# v = dict_data[key_del]
# for i in range(len(dict_data[key_del]['H'])):
#     if v['name'][i] != item_del:
#         tmp_key['H'].append(v['H'][i])
#         tmp_key['S'].append(v['S'][i])
#         tmp_key['V'].append(v['V'][i])
#         tmp_key['D'].append(v['D'][i])
#         name = '{0}_{1}'.format(kind, len(tmp_key['H']))
#         tmp_key['name'].append(name)
#
# dict_data[key_del] = tmp_key

In [21]:
key_list.stop()
mouse_list.stop()

In [22]:
cv2.destroyAllWindows()

In [23]:
import minerl
import gym
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np, cv2
from pynput.keyboard import Key, Listener
from pynput.mouse import Button
from pynput import mouse
from threading import Thread
import pickle
import time
import random
import imutils
from pairing import pair, depair
import torch
torch.cuda.set_device(0)
# class PixelConnect():
#     def __init__(self):
#         super.__init__()
#
# class CompareNeighbor():
#     def __init__(self, max_neighbor_pixel=1, image_size=64):
#         super.__init__()
#         x = np.linspace(-max_neighbor_pixel, max_neighbor_pixel, max_neighbor_pixel * 2 + 1)
#         self.neighbor_pixels =  [[], []]
#         for i in range(len(x)):
#             for j in range(len(y)):
#                 if x[i] != 0 and x[j] != 0:
#                     self.neighbor_pixels[0].append(x[i])
#                     self.neighbor_pixels[1].append(x[j])
#
#
#         x = np.arange(1, image_size, 2)
#         self.pixels_map = {}
#         for i in range(len(x)):
#             for j in range(len(x)):
#                 self.pixels_map[tuple((x[i], x[j]))] = PixelConnect()
#
#     def make_pixel_connect(self, image):
#         hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
#         for pixel in self.pixels_map:
#

a = 3

# obs = env.reset()


# env = gym.make('MineRLObtainDiamond-v0')
# obs = env.reset()
# record_p = []

# hsv_bond = {}
#
# key_list.stop()
# mouse_list.stop()




def nothing(x):
    pass


def trafer_hsv(dict):
    cv2.setTrackbarPos('HMin', 'image', dict['hMin'])
    cv2.setTrackbarPos('SMin', 'image', dict['sMin'])
    if 'vMin' in dict:
        cv2.setTrackbarPos('VMin', 'image', dict['vMin'])
    else:
        cv2.setTrackbarPos('VMin', 'image', 0)

    cv2.setTrackbarPos('HMax', 'image', dict['hMax'])
    cv2.setTrackbarPos('SMax', 'image', dict['sMax'])

    if 'vMax' in dict:
        cv2.setTrackbarPos('VMax', 'image', dict['vMax'])
    else:
        cv2.setTrackbarPos('VMax', 'image', 255)


def on_move(x, y):
    global attack_b


def on_click(x, y, button, pressed):
    global attack_b
    global action
    # print('{0} at {1}'.format(
    #     button if pressed else 'Released',
    #     (x, y)))
    if pressed:
        try:
            if button == Button.left:
                action['attack'] = 1
        except AttributeError:
            pass
    else:
        action['attack'] = 0
    global key_on_release
    if key_on_release == 'esc':
        return False


def on_scroll(x, y, dx, dy):
    global attack_b


def on_press(key):
    global key_on_press
    global action
    try:
        key_on_press = key.char
        if key_on_press == 'w' and action['back'] != 1:
            action['forward'] = 1

        if key_on_press == 's' and action['forward'] != 1:
            action['back'] = 1

        if key_on_press == 'a' and action['right'] != 1:
            action['left'] = 1
        if key_on_press == 'd' and action['left'] != 1:
            action['right'] = 1

        if key_on_press == 'q' and action['camera'][1] == 0:
            action['camera'][1] = -15

        if key_on_press == 'e' and action['camera'][1] == 0:
            action['camera'][1] = 15

        if key_on_press == 'r' and action['camera'][0] == 0:
            action['camera'][0] = -15
        if key_on_press == 'f' and action['camera'][0] == 0:
            action['camera'][0] = 15


    except AttributeError:
        # do something when a certain key is pressed, using key, not key.char
        pass
    if key == Key.space:
        action['jump'] = 1


def on_release(key):
    global key_on_release
    global jump
    global action
    global pasue
    global lock
    try:
        key_on_release = key.char
        if key_on_release == 'w':
            action['forward'] = 0
        if key_on_release == 's':
            action['back'] = 0
        if key_on_release == 'a':
            action['left'] = 0
        if key_on_release == 'd':
            action['right'] = 0

        if key_on_release == 'q':
            action['camera'][1] = 0
        if key_on_release == 'e':
            action['camera'][1] = 0
        if key_on_release == 'r':
            action['camera'][0] = 0
        if key_on_release == 'f':
            action['camera'][0] = 0
        if key_on_release == 'p':
            pasue = not pasue
            print("pause ", pasue)
        if key_on_release == 'l':
            lock = not lock
            print(lock)
        # if key.char == 'o':
        #     return False
    except AttributeError:
        # do something when a certain key is pressed, using key, not key.char
        pass

    if key == Key.esc:
        # Stop listener
        print("Stop lister ")
        key_on_release = 'esc'
        return False
    if key == Key.space:
        action['jump'] = 0

def shift_image(X, dx, dy):
    X = np.array(X, dtype=np.float64)
    X = np.roll(X, dy, axis=0)
    X = np.roll(X, dx, axis=1)
    if dy>0:
        X[:dy, :] = 1500
    elif dy<0:
        X[dy:, :] = 1500
    if dx>0:
        X[:, :dx] = 1500
    elif dx<0:
        X[:, dx:] = 1500
    return X

def Life2CodingRGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  # checks mouse moves
        global hsv
        #image_2 = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        colorsBGR = hsv[y, x]
        # colorsRGB=cv2.cvtColor(colorsBGR,cv2.COLOR_BGR2HSV)
        global tmp_33
        global names
        global dict_data
        global h_range
        global s_range
        global v_range
        global X_VALUE
        global Y_VALUE
        X_VALUE = y
        Y_VALUE = x
        H_VALUE = colorsBGR[0]
        S_VALUE = colorsBGR[1]
        V_VALUE = colorsBGR[2]
        index = tmp_33[y,x]

        print("RGB Value at ({},{}):{} {} {}".format(x,y,colorsBGR, names[index],
                                                     [h_range[y,x,index],
                                                      s_range[y,x,index],
                                                      v_range[y,x,index]]), end='\r')

def Life2CodingRGB2(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  # checks mouse moves
        image_2  =cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        colorsBGR = image_2[y, x]
        # colorsRGB=cv2.cvtColor(colorsBGR,cv2.COLOR_BGR2HSV)
        global tmp_33
        global names
        print("RGB Value at ({},{}):{} {}".format(x,y,colorsBGR, names[tmp_33[y,x]]), end='\r')


cv2.namedWindow('mask', cv2.WINDOW_KEEPRATIO)
cv2.resizeWindow('mask', 1000, 1000)
cv2.moveWindow('mask', 1100, 30)
cv2.namedWindow('image', cv2.WINDOW_KEEPRATIO)
cv2.resizeWindow('image', 1000, 1000)
cv2.moveWindow('image', -30, 30)



cv2.namedWindow('color_range')
cv2.moveWindow('color_range', 40, 1000)

# cv2.createTrackbar('HMin', 'image', 0, 179, nothing)
# cv2.createTrackbar('SMin', 'image', 0, 255, nothing)
# cv2.createTrackbar('HMax', 'image', 0, 179, nothing)
# cv2.createTrackbar('SMax', 'image', 0, 255, nothing)
# cv2.createTrackbar('VMin', 'image', 0, 255, nothing)
# cv2.createTrackbar('VMax', 'image', 0, 255, nothing)
# # Set default value for Max HSV trackbars
# cv2.setTrackbarPos('HMax', 'image', 179)
# cv2.setTrackbarPos('SMax', 'image', 255)
# cv2.setTrackbarPos('VMax', 'image', 255)

# Initialize HSV min/max values
hMin = sMin = vMin = hMax = sMax = vMax = 0
phMin = psMin = pvMin = phMax = psMax = pvMax = 0

index = 1
action = env.action_space.noop()
key_list = Listener(on_press=on_press, on_release=on_release)
mouse_list = mouse.Listener(on_move=on_move, on_click=on_click, on_scroll=on_scroll)
attack_b = False
key_list.start()
mouse_list.start()

key_on_press = ''
key_on_release = ''
pasue = False
lock = True
levels = [-1, -1, -1]
current_level = -1
indexs = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

pool_size = 20
pool_colors = np.zeros([pool_size,3])
np.random.seed(10)
x = list(np.arange(0, 179, 179/pool_size))
pool_colors[:,0] =np.random.permutation(x)
pool_colors[:,1] = np.random.choice(range(50, 200), size=pool_size)
pool_colors[:,2] = np.random.choice(range(100, 200), size=pool_size)

names_stack = []
t_R = 0
font = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (400, 400)
fontScale = 0.6
fontColor = (0, 0, 255)
lineType = 1

max_size = 128 * 15

x = np.linspace(-1, 1, 3, dtype=np.int64)
# x = [-2, -1, 1, 2]
# neighbor_pixels = [[-1, 0, 1, 0, -2, 2, 2, -2], [0, 1, 0, -1, -2, -2, 2, 2]]
neighbor_pixels = [[], []]
for i in range(len(x)):
    for j in range(len(x)):
        if not (x[i] == 0 and x[j] == 0):
            neighbor_pixels[0].append(x[i])
            neighbor_pixels[1].append(x[j])
neighbor_pixels = np.array(neighbor_pixels)
H_VALUE = 0
S_VALUE = 0
V_VALUE = 0
X_VALUE = 0
Y_VALUE = 0
dict_data = {}
with open('data_1.pkl', 'rb') as f:
    dict_data = pickle.load(f)

h_range = []
s_range = []
v_range = []

names = []
kind = []
tree_chunk = []
for k, v in dict_data.items():
    for i in range(len(v['H'])):
        h_range.append(v['H'][i])
        s_range.append(v['S'][i])
        v_range.append(v['V'][i])

        names.append(v['name'][i])
        kind.append(k)
        if k == "tree_chunk":
            tree_chunk.append(255)
        else:
            tree_chunk.append(0)

x = np.sort(h_range)
h_range = np.expand_dims(h_range, axis=0)
h_range = np.repeat(h_range, 64, axis=0)
h_range = np.expand_dims(h_range, axis=0)
h_range = np.repeat(h_range, 64, axis=0)
h_range = np.array(h_range, dtype=np.float64)
h_range = torch.Tensor(h_range)

s_range = np.expand_dims(s_range, axis=0)
s_range = np.repeat(s_range, 64, axis=0)
s_range = np.expand_dims(s_range, axis=0)
s_range = np.repeat(s_range, 64, axis=0)
s_range = np.array(s_range, dtype=np.float64)
s_range = torch.Tensor(s_range)

v_range = np.expand_dims(v_range, axis=0)
v_range = np.repeat(v_range, 64, axis=0)
v_range = np.expand_dims(v_range, axis=0)
v_range = np.repeat(v_range, 64, axis=0)
v_range = np.array(v_range, dtype=np.float64)
v_range = torch.Tensor(v_range)



tree_chunk = np.array(tree_chunk, dtype=np.uint8)

keyboard_map = {'c': 'tree_chunk', 't': 'tree_leave', 'r': 'rock', 's': 'sand', 'd': 'dirt',
                'u': 'not_common', 'g':'grass','b':'black','w': 'water', 'k': 'sky', 'n':'snow',
                'm':'monster', 'a':'animal', 'h': 'coal', 'i': 'iron'}
key_selected = ''
add_done = False
while (True):
    # Display result image
    color_range = np.zeros([128, max_size, 3], dtype=np.uint8)
    tmp_time = time.time()

    if pasue:
        # action['place'] = 'furnace'
        # action['place'] = 'torch'

        # action['nearbyCraft'] = 'iron_pickaxe'
        # action['nearbyCraft'] = 'stone_pickaxe'
        action = model.predict(obs.copy())
        obs, reward, done, _ = env.step(action)
        t_R += reward
        # obs['inventory']
        # print(obs['inventory']['iron_ore'], end='\r')

        # record_p.append(obs['pov'])
    else:
        action = 0
    # if action['jump'] == 1:
    #     action['jump'] = 0
    image = obs['pov']

    # Convert to HSV format and color threshold
    X = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)


    tmp_mask = np.zeros([64, 64, 3], dtype=np.uint8)

    hsv_64 = np.array(hsv, dtype=np.float64)
    hsv_64 = torch.Tensor(hsv_64)
    pixels_maps = {}
    keys_mask = {}
    key_index = 0

    H_nei = np.zeros((64, 64, len(neighbor_pixels[0])), dtype=np.float64)
    S_nei = np.zeros((64, 64, len(neighbor_pixels[0])), dtype=np.float64)
    V_nei = np.zeros((64, 64, len(neighbor_pixels[0])), dtype=np.float64)

    for counter in range(len(neighbor_pixels[0])):
        new_hsv = shift_image(hsv_64, neighbor_pixels[0][counter],
                              neighbor_pixels[1][counter])
        # zero_index = np.where(new_hsv[:, :, 0] == 1500)
        # new_hsv[zero_index[0], zero_index[1], :] =
        new_hsv = np.abs(hsv_64 - new_hsv)
        index = np.where(new_hsv[:,:,0] >= 89.5)
        new_hsv[index[0], index[1], 0] = np.abs(179 - new_hsv[index[0], index[1], 0])
        H_nei[:, :, counter] = new_hsv[:, :, 0]
        S_nei[:, :, counter] = new_hsv[:, :, 1]
        V_nei[:, :, counter] = new_hsv[:, :, 1]


    # for i in range(64):
    #     for j in range(64):
    #         pixel_id = tuple((i, j))
    #         pixel_hsv = hsv_64[i, j, :]
    #         H_error = H_nei[i, j]
    #         H_error_min = np.min(H_error)
    #         S_error = S_nei[i, j]
    #         V_error = V_nei[i, j]
    #
    #         if H_error_min <= 1:
    #             key = None
    #             for ins in range(len(H_error)):
    #                 next_pixel = tuple((i - neighbor_pixels[1][ins],
    #                                     j - neighbor_pixels[0][ins]))
    #
    #                 if H_error[ins] <= 1 and S_error[ins] <= 1 and V_error[ins] <= 1:
    #                     # if next_pixel[0] < 0:
    #                     #     print(i, j, neighbor_pixels[0][ins])
    #                     if key is None:
    #                         if pixel_id in pixels_maps:
    #                             key = pixels_maps[pixel_id]
    #
    #                         elif next_pixel in pixels_maps:
    #                             key = pixels_maps[next_pixel]
    #                         else:
    #                             key = key_index
    #                             keys_mask[key] = [[], [], {}, [], []]
    #                             key_index += 1
    #                     # print(key)
    #                     pixels_maps[pixel_id] = key
    #                     pixels_maps[next_pixel] = key
    #                     if pixel_id not in keys_mask[key][2]:
    #                         keys_mask[key][2][pixel_id] = 0
    #                         keys_mask[key][0].append(pixel_id[0])
    #                         keys_mask[key][1].append(pixel_id[1])
    #
    #                     if next_pixel not in keys_mask[key][2]:
    #                         keys_mask[key][2][next_pixel] = 0
    #                         keys_mask[key][0].append(next_pixel[0])
    #                         keys_mask[key][1].append(next_pixel[1])

    # # print(len(keys_mask))
    # max = 0
    # k_m = 0
    # for k, v in keys_mask.items():
    #     hsv_64[v[0], v[1], 0] = np.mean(hsv[v[0], v[1], 0])
    #     hsv_64[v[0], v[1], 1] = np.mean(hsv[v[0], v[1], 1])
    #     hsv_64[v[0], v[1], 2] = np.mean(hsv[v[0], v[1], 2])
    #     # hsv[v[3], v[4], 0] = np.mean(hsv[v[3], v[4], 0])
    #     # hsv[v[3], v[4], 1] = np.mean(hsv[v[3], v[4], 1])
    #     # tmp_mask[v[3], v[4], 2] = np.mean(hsv[v[3], v[4], 2]) + 20
    # len(keys_mask)
    tmp_time = time.time()


    h = np.expand_dims(hsv_64[:, :, 0], axis=-1)
    s = np.expand_dims(hsv_64[:, :, 1], axis=-1)
    v = np.expand_dims(hsv_64[:, :, 2], axis=-1)
    h = torch.unsqueeze(hsv_64[:,:,0], -1)
    s = torch.unsqueeze(hsv_64[:,:,1], -1)
    v = torch.unsqueeze(hsv_64[:,:,2], -1)
    #
    # h = torch.Tensor(h)
    # s = torch.Tensor(s)
    # v = torch.Tensor(v)
    v.shape
    # H1_range.shape

    if len(dict_data['rock']['H']) != 0:

        h_error = torch.abs(h - h_range)
        index = torch.where(h_error >= 89.5)
        h_error[index[0], index[1], index[2]] = 179 - h_error[index[0], index[1], index[2]]


        tmp_33 = torch.sqrt(torch.pow(h_error, 2) + torch.pow((s/4 - s_range/4), 2)
                            + torch.pow((v/6 - v_range/6), 2))

        tmp_33 = torch.argmin(tmp_33, axis=2)
        # print(time.time() - tmp_time)
        x = tree_chunk[tmp_33]
        # tmp_image = np.expand_dims(tmp_image,axis=-1)
        x = np.expand_dims(x,axis=-1)
        x.shape
        index_color = 0
        for i in range(64):
            for j in range(64):
                pixel = hsv_64[i, j, :]
                index_x = tmp_33[i,j]
                kins = kind[index_x]
                # if kins not in kindss:
                #     kindss[kins] = index_color
                #     index_color += 1

                # tmp_mask[i, j, :] = np.array([hsv_r[names[index_x]]['H'],
                #                               hsv_r[names[index_x]]['S'],
                #                               hsv_r[names[index_x]]['v']], dtype=np.uint8)

                if kins in names_stack:
                    index_1 = np.where(np.array(names_stack) == kins)[0][0]
                else:
                    names_stack.append(kins)
                    index_1 = len(names_stack)
                color = pool_colors[index_1]
                tmp_mask[i, j, :] = color

                if (index_1 + 1) * 128 < max_size:
                    if color_range[0, index_1 * 128 +10, 0] == 0:
                        color_range[:, index_1 * 128: (index_1 + 1) * 128, :] = color
                        cv2.putText(color_range, kins,
                                    (index_1 * 128 + 20, 64),
                                    font,
                                    fontScale,
                                    fontColor,
                                    lineType)
        # print(time.time() - tmp_time)

    # tmp_33[29,22]
    if not lock and not pasue:
        a = 3
        if key_on_press in keyboard_map and key_on_release != 'l':
            key_selected = keyboard_map[key_on_press]
            print("new: key {0} - {1} - {2}".format(key_selected, [hsv_64[X_VALUE,Y_VALUE, 0],
                                                                   hsv_64[X_VALUE,Y_VALUE, 1],
                                                                   hsv_64[X_VALUE,Y_VALUE, 2]], [X_VALUE, Y_VALUE]))
            key_on_press = ''
            add_done = False

        if key_selected != '':
            if key_on_press in indexs:
                if int(key_on_press) == 1:
                    dict_data[key_selected]['H'].append(hsv_64[X_VALUE,Y_VALUE,0])
                    dict_data[key_selected]['S'].append(hsv_64[X_VALUE,Y_VALUE,1])
                    dict_data[key_selected]['V'].append(hsv_64[X_VALUE,Y_VALUE,2])

                    name = '{0}_{1}'.format(key_selected, len(dict_data[key_selected]['H']))

                    dict_data[key_selected]['name'].append(name)

                    print("ADDED : key {0} - {1} - {2}".format(key_selected, [H_VALUE, S_VALUE, V_VALUE],
                                                               [X_VALUE, Y_VALUE]))
                    with open('data_1.pkl', 'wb') as f:
                        pickle.dump(dict_data, f)


                if int(key_on_press) == 2:
                    print("STOP")
                    key_selected = ''
                    add_done = False
                if int(key_on_press) == 4:
                    h_del = dict_data[key_selected]['H'][-1]
                    s_del = dict_data[key_selected]['S'][-1]
                    v_del = dict_data[key_selected]['V'][-1]
                    name_del = dict_data[key_selected]['name'][-1]
                    dict_data[key_selected]['H'] = dict_data[key_selected]['H'][:-1]
                    dict_data[key_selected]['S'] = dict_data[key_selected]['S'][:-1]
                    dict_data[key_selected]['V'] = dict_data[key_selected]['V'][:-1]
                    dict_data[key_selected]['name'] = dict_data[key_selected]['name'][:-1]


                    print("DEL : key {0} - {1}".format(key_selected, [h_del, s_del, v_del]))
                    # key_selected = ''
                    add_done = False
                    with open('data_1.pkl', 'wb') as f:
                        pickle.dump(dict_data, f)

                h_range = []
                s_range = []
                v_range = []

                names = []
                kind = []
                for k, v in dict_data.items():
                    for i in range(len(v['H'])):
                        h_range.append(v['H'][i])
                        s_range.append(v['S'][i])
                        v_range.append(v['V'][i])

                        names.append(v['name'][i])
                        kind.append(k)
                print(len(h_range))

                h_range = np.expand_dims(h_range, axis=0)
                h_range = np.repeat(h_range, 64, axis=0)
                h_range = np.expand_dims(h_range, axis=0)
                h_range = np.repeat(h_range, 64, axis=0)
                h_range = np.array(h_range, dtype=np.float64)
                h_range = torch.Tensor(h_range)

                s_range = np.expand_dims(s_range, axis=0)
                s_range = np.repeat(s_range, 64, axis=0)
                s_range = np.expand_dims(s_range, axis=0)
                s_range = np.repeat(s_range, 64, axis=0)
                s_range = np.array(s_range, dtype=np.float64)
                s_range = torch.Tensor(s_range)

                v_range = np.expand_dims(v_range, axis=0)
                v_range = np.repeat(v_range, 64, axis=0)
                v_range = np.expand_dims(v_range, axis=0)
                v_range = np.repeat(v_range, 64, axis=0)
                v_range = np.array(v_range, dtype=np.float64)
                v_range = torch.Tensor(v_range)

                add_done = True
                key_on_press = ''
        #image = cv2.imread("./block_image/Mega_Spruce_Tree.png")

    # print(np.count_nonzero(names == 32)/4096, end='\r')
    # mask = 255 - mask
    # hsv[:,:,2] = 255
    # X = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)



    # X = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

    # dst = cv2.addWeighted(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), 0.7,
    #                       cv2.cvtColor(tmp_mask, cv2.COLOR_HSV2RGB), 0.7, 0)
    #hsv = np.array(hsv_64,dtype=np.uint8)
    # hsv[48,19,0] = hsv[48,18,0]
    # hsv[48,19,1] = hsv[48,18,1]
    #hsv[48,19,2]=  hsv[48,18,2]

    tmp_hsv = np.array(hsv)
    #tmp_hsv[X_VALUE,Y_VALUE,0] = 50

    tmp_hsv[X_VALUE,Y_VALUE,1] = 255

    tmp_hsv[X_VALUE,Y_VALUE,2] = 230
    #tmp_hsv[:,:,2] = 255

    # index = np.where((hsv[:, :, 0] >=20) |( hsv[:, :, 0] <= 16))
    #hsv[index[0], index[1], :] = [0,0, 255]
    # tmp_hsv[:,:,0] = 179
    # tmp_hsv[:,:,1] = 2
    # tmp_hsv[:,:,2] = tmp_hsv[:,:,2] + 50

    tmp_image = cv2.cvtColor(tmp_hsv, cv2.COLOR_HSV2BGR)
    # tmp_image = cv2.cvtColor(tmp_image, cv2.COLOR_BGR2GRAY)

    # tmp_image[0,1]
    # tmp_image[X_VALUE,Y_VALUE,:] = [100,100,100]
    tmp_mask[X_VALUE,Y_VALUE,2] = 230

    cv2.imshow('image',tmp_image)
    cv2.resizeWindow('image', 950, 950)
    cv2.setMouseCallback('image', Life2CodingRGB)

    cv2.imshow('mask', cv2.cvtColor(tmp_mask, cv2.COLOR_HSV2RGB))
    cv2.resizeWindow('mask', 950, 950)
    cv2.setMouseCallback('mask', Life2CodingRGB)


    cv2.imshow('color_range', cv2.cvtColor(color_range, cv2.COLOR_HSV2RGB))
    # cv2.resizeWindow('mask', 800, 800)

    # for c in cnts:
    #     x, y, w, h = cv2.boundingRect(c)
    #
    #     print(cv2.contourArea(c))
    # print("---")
    # image = obs['pov']
    # cv2.imshow('image', cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # print(time.time() - tmp_time)

    if cv2.waitKey(10) & 0xFF == ord('o'):
        break
    # time.sleep(0.1)

key_list.stop()
mouse_list.stop()

cv2.destroyAllWindows()
hsv[16,60]
# with open('data_1.pkl', 'wb') as f:
#     pickle.dump(dict_data, f)


# pixels_maps = {}
# keys_mask = {}
# key_index = 0
# hsv_64 = np.array(hsv, dtype=np.float64)
# for i in range(64):
#     for j in range(64):
#
#         pixel_id = tuple((i, j))
#         pixel_hsv = hsv_64[i, j, :]
#
#         neigh_bor = [neighbor_pixels[0] + i, neighbor_pixels[1] + j]
#         index = np.where(((neigh_bor[0] <= 63) & (neigh_bor[0] > 0)))
#         neigh_bor[0] = neigh_bor[0][index]
#         neigh_bor[1] = neigh_bor[1][index]
#         index = np.where(((neigh_bor[1] <= 63) & (neigh_bor[1] > 0)))
#         neigh_bor[0] = neigh_bor[0][index]
#         neigh_bor[1] = neigh_bor[1][index]
#
#         neigh_hs = hsv_64[neigh_bor[0], neigh_bor[1], : 2]
#
#         HS_error = np.sum(np.abs(neigh_hs - pixel_hsv[:2]), axis=1) / 2
#         HS_error_min = np.min(HS_error)
#         if HS_error_min <= 25:
#             for ins in range(len(HS_error)):
#                 if np.abs(HS_error[ins] - HS_error_min) <= 5:
#                     next_pixel = tuple((neigh_bor[0][ins], neigh_bor[1][ins]))
#                     if next_pixel in pixels_maps:
#                         key = pixels_maps[next_pixel]
#                     else:
#                         key = key_index
#                         keys_mask[key] = [[], [], {}]
#                         key_index += 1
#                     pixels_maps[pixel_id] = key
#                     pixels_maps[next_pixel] = key
#                     if pixel_id not in keys_mask[key][2]:
#                         keys_mask[key][2][pixel_id] = 0
#                         keys_mask[key][0].append(pixel_id[0])
#                         keys_mask[key][1].append(pixel_id[1])
#                     if next_pixel not in keys_mask[key][2]:
#                         keys_mask[key][2][next_pixel] = 0
#                         keys_mask[key][0].append(next_pixel[0])
#                         keys_mask[key][1].append(next_pixel[1])
# keys_mask[0]
# for k,v in keys_mask.items():
#     print(len(v[0]))
#     # tmp_mask[v[0], v[1],0] = np.mean(hsv[v[0], v[1],0])
#     # tmp_mask[v[0], v[1],1] = np.mean(hsv[v[0], v[1],1])
#     # tmp_mask[v[0], v[1],2] = np.mean(hsv[v[0], v[1],2])
a = 3
a = 3
# dict_data['dirt']
# dict_data['tree_chunk']

# dict_data = {}
# for k,v in keyboard_map.items():
#     kind = v
#     if kind not in dict_data:
#         dict_data[kind] = {'H': [], 'S': [], 'V': [],
#                            'H_1': [], 'S_1': [],
#                            'H_2': [], 'S_2': [],
#                            'name': []}
# with open('data_1.pkl', 'wb') as f:
#     pickle.dump(dict_data, f)
# dict_data[kind]['H'].append(v['H'])
# dict_data[kind]['S'].append(v['S'])
# dict_data[kind]['V'].append(v['v'])
# name = '{0}_{1}'.format(kind, len(dict_data[kind]['H']))
# dict_data[kind]['name'].append(name)
#


# with open('data.pkl', 'rb') as f:
#     output = pickle.load(f)
#
# h_range = []
# s_range = []
# v_range = []
# names = []
# kind = []
# for k,v in output.items():
#     for i in range(len(v['H'])):
#         h_range.append(v['H'][i])
#         s_range.append(v['S'][i])
#         v_range.append(v['V'][i])
#         names.append(v['name'][i])
#         kind.append(k)
# h_range = np.expand_dims(h_range,axis=0)
# h_range = np.repeat(h_range, 64, axis=0)
# h_range = np.expand_dims(h_range,axis=0)
# h_range = np.repeat(h_range, 64, axis=0)
#
# s_range = np.expand_dims(s_range,axis=0)
# s_range = np.repeat(s_range, 64, axis=0)
# s_range = np.expand_dims(s_range,axis=0)
# s_range = np.repeat(s_range, 64, axis=0)
# h_range = np.array(h_range, dtype=np.float64)
# s_range = np.array(s_range, dtype=np.float64)
#
# v_range = np.expand_dims(v_range,axis=0)
# v_range = np.repeat(v_range, 64, axis=0)
# v_range = np.expand_dims(v_range,axis=0)
# v_range = np.repeat(v_range, 64, axis=0)
# v_range = np.array(v_range, dtype=np.float64)

# key_del = 'tree_chunk'
# item_del = 'tree_chunk_4'
# tmp_key = {'H': [], 'S': [], 'V': [],
#                            'D': [],
#                            'name': []}
# v = dict_data[key_del]
# for i in range(len(dict_data[key_del]['H'])):
#     if v['name'][i] != item_del:
#         tmp_key['H'].append(v['H'][i])
#         tmp_key['S'].append(v['S'][i])
#         tmp_key['V'].append(v['V'][i])
#         tmp_key['D'].append(v['D'][i])
#         name = '{0}_{1}'.format(kind, len(tmp_key['H']))
#         tmp_key['name'].append(name)
#
# dict_data[key_del] = tmp_key

In [24]:
key_list.stop()
mouse_list.stop()

In [25]:
import minerl
import gym
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np, cv2
from pynput.keyboard import Key, Listener
from pynput.mouse import Button
from pynput import mouse
from threading import Thread
import pickle
import time
import random
import imutils
from pairing import pair, depair
import torch
torch.cuda.set_device(0)
# class PixelConnect():
#     def __init__(self):
#         super.__init__()
#
# class CompareNeighbor():
#     def __init__(self, max_neighbor_pixel=1, image_size=64):
#         super.__init__()
#         x = np.linspace(-max_neighbor_pixel, max_neighbor_pixel, max_neighbor_pixel * 2 + 1)
#         self.neighbor_pixels =  [[], []]
#         for i in range(len(x)):
#             for j in range(len(y)):
#                 if x[i] != 0 and x[j] != 0:
#                     self.neighbor_pixels[0].append(x[i])
#                     self.neighbor_pixels[1].append(x[j])
#
#
#         x = np.arange(1, image_size, 2)
#         self.pixels_map = {}
#         for i in range(len(x)):
#             for j in range(len(x)):
#                 self.pixels_map[tuple((x[i], x[j]))] = PixelConnect()
#
#     def make_pixel_connect(self, image):
#         hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
#         for pixel in self.pixels_map:
#

a = 3

# obs = env.reset()


# env = gym.make('MineRLObtainDiamond-v0')
# obs = env.reset()
# record_p = []

# hsv_bond = {}
#
key_list.stop()
mouse_list.stop()




def nothing(x):
    pass


def trafer_hsv(dict):
    cv2.setTrackbarPos('HMin', 'image', dict['hMin'])
    cv2.setTrackbarPos('SMin', 'image', dict['sMin'])
    if 'vMin' in dict:
        cv2.setTrackbarPos('VMin', 'image', dict['vMin'])
    else:
        cv2.setTrackbarPos('VMin', 'image', 0)

    cv2.setTrackbarPos('HMax', 'image', dict['hMax'])
    cv2.setTrackbarPos('SMax', 'image', dict['sMax'])

    if 'vMax' in dict:
        cv2.setTrackbarPos('VMax', 'image', dict['vMax'])
    else:
        cv2.setTrackbarPos('VMax', 'image', 255)


def on_move(x, y):
    global attack_b


def on_click(x, y, button, pressed):
    global attack_b
    global action
    # print('{0} at {1}'.format(
    #     button if pressed else 'Released',
    #     (x, y)))
    if pressed:
        try:
            if button == Button.left:
                action['attack'] = 1
        except AttributeError:
            pass
    else:
        action['attack'] = 0
    global key_on_release
    if key_on_release == 'esc':
        return False


def on_scroll(x, y, dx, dy):
    global attack_b


def on_press(key):
    global key_on_press
    global action
    try:
        key_on_press = key.char
        if key_on_press == 'w' and action['back'] != 1:
            action['forward'] = 1

        if key_on_press == 's' and action['forward'] != 1:
            action['back'] = 1

        if key_on_press == 'a' and action['right'] != 1:
            action['left'] = 1
        if key_on_press == 'd' and action['left'] != 1:
            action['right'] = 1

        if key_on_press == 'q' and action['camera'][1] == 0:
            action['camera'][1] = -15

        if key_on_press == 'e' and action['camera'][1] == 0:
            action['camera'][1] = 15

        if key_on_press == 'r' and action['camera'][0] == 0:
            action['camera'][0] = -15
        if key_on_press == 'f' and action['camera'][0] == 0:
            action['camera'][0] = 15


    except AttributeError:
        # do something when a certain key is pressed, using key, not key.char
        pass
    if key == Key.space:
        action['jump'] = 1


def on_release(key):
    global key_on_release
    global jump
    global action
    global pasue
    global lock
    try:
        key_on_release = key.char
        if key_on_release == 'w':
            action['forward'] = 0
        if key_on_release == 's':
            action['back'] = 0
        if key_on_release == 'a':
            action['left'] = 0
        if key_on_release == 'd':
            action['right'] = 0

        if key_on_release == 'q':
            action['camera'][1] = 0
        if key_on_release == 'e':
            action['camera'][1] = 0
        if key_on_release == 'r':
            action['camera'][0] = 0
        if key_on_release == 'f':
            action['camera'][0] = 0
        if key_on_release == 'p':
            pasue = not pasue
            print("pause ", pasue)
        if key_on_release == 'l':
            lock = not lock
            print(lock)
        # if key.char == 'o':
        #     return False
    except AttributeError:
        # do something when a certain key is pressed, using key, not key.char
        pass

    if key == Key.esc:
        # Stop listener
        print("Stop lister ")
        key_on_release = 'esc'
        return False
    if key == Key.space:
        action['jump'] = 0

def shift_image(X, dx, dy):
    X = np.array(X, dtype=np.float64)
    X = np.roll(X, dy, axis=0)
    X = np.roll(X, dx, axis=1)
    if dy>0:
        X[:dy, :] = 1500
    elif dy<0:
        X[dy:, :] = 1500
    if dx>0:
        X[:, :dx] = 1500
    elif dx<0:
        X[:, dx:] = 1500
    return X

def Life2CodingRGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  # checks mouse moves
        global hsv
        #image_2 = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        colorsBGR = hsv[y, x]
        # colorsRGB=cv2.cvtColor(colorsBGR,cv2.COLOR_BGR2HSV)
        global tmp_33
        global names
        global dict_data
        global h_range
        global s_range
        global v_range
        global X_VALUE
        global Y_VALUE
        X_VALUE = y
        Y_VALUE = x
        H_VALUE = colorsBGR[0]
        S_VALUE = colorsBGR[1]
        V_VALUE = colorsBGR[2]
        index = tmp_33[y,x]

        print("RGB Value at ({},{}):{} {} {}".format(x,y,colorsBGR, names[index],
                                                     [h_range[y,x,index],
                                                      s_range[y,x,index],
                                                      v_range[y,x,index]]), end='\r')

def Life2CodingRGB2(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  # checks mouse moves
        image_2  =cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        colorsBGR = image_2[y, x]
        # colorsRGB=cv2.cvtColor(colorsBGR,cv2.COLOR_BGR2HSV)
        global tmp_33
        global names
        print("RGB Value at ({},{}):{} {}".format(x,y,colorsBGR, names[tmp_33[y,x]]), end='\r')


cv2.namedWindow('mask', cv2.WINDOW_KEEPRATIO)
cv2.resizeWindow('mask', 1000, 1000)
cv2.moveWindow('mask', 1100, 30)
cv2.namedWindow('image', cv2.WINDOW_KEEPRATIO)
cv2.resizeWindow('image', 1000, 1000)
cv2.moveWindow('image', -30, 30)



cv2.namedWindow('color_range')
cv2.moveWindow('color_range', 40, 1000)

# cv2.createTrackbar('HMin', 'image', 0, 179, nothing)
# cv2.createTrackbar('SMin', 'image', 0, 255, nothing)
# cv2.createTrackbar('HMax', 'image', 0, 179, nothing)
# cv2.createTrackbar('SMax', 'image', 0, 255, nothing)
# cv2.createTrackbar('VMin', 'image', 0, 255, nothing)
# cv2.createTrackbar('VMax', 'image', 0, 255, nothing)
# # Set default value for Max HSV trackbars
# cv2.setTrackbarPos('HMax', 'image', 179)
# cv2.setTrackbarPos('SMax', 'image', 255)
# cv2.setTrackbarPos('VMax', 'image', 255)

# Initialize HSV min/max values
hMin = sMin = vMin = hMax = sMax = vMax = 0
phMin = psMin = pvMin = phMax = psMax = pvMax = 0

index = 1
# action = env.action_space.noop()
key_list = Listener(on_press=on_press, on_release=on_release)
mouse_list = mouse.Listener(on_move=on_move, on_click=on_click, on_scroll=on_scroll)
attack_b = False
key_list.start()
mouse_list.start()

key_on_press = ''
key_on_release = ''
pasue = False
lock = True
levels = [-1, -1, -1]
current_level = -1
indexs = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

pool_size = 20
pool_colors = np.zeros([pool_size,3])
np.random.seed(10)
x = list(np.arange(0, 179, 179/pool_size))
pool_colors[:,0] =np.random.permutation(x)
pool_colors[:,1] = np.random.choice(range(50, 200), size=pool_size)
pool_colors[:,2] = np.random.choice(range(100, 200), size=pool_size)

names_stack = []
t_R = 0
font = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (400, 400)
fontScale = 0.6
fontColor = (0, 0, 255)
lineType = 1

max_size = 128 * 15

x = np.linspace(-1, 1, 3, dtype=np.int64)
# x = [-2, -1, 1, 2]
# neighbor_pixels = [[-1, 0, 1, 0, -2, 2, 2, -2], [0, 1, 0, -1, -2, -2, 2, 2]]
neighbor_pixels = [[], []]
for i in range(len(x)):
    for j in range(len(x)):
        if not (x[i] == 0 and x[j] == 0):
            neighbor_pixels[0].append(x[i])
            neighbor_pixels[1].append(x[j])
neighbor_pixels = np.array(neighbor_pixels)
H_VALUE = 0
S_VALUE = 0
V_VALUE = 0
X_VALUE = 0
Y_VALUE = 0
dict_data = {}
with open('data_1.pkl', 'rb') as f:
    dict_data = pickle.load(f)

h_range = []
s_range = []
v_range = []

names = []
kind = []
tree_chunk = []
for k, v in dict_data.items():
    for i in range(len(v['H'])):
        h_range.append(v['H'][i])
        s_range.append(v['S'][i])
        v_range.append(v['V'][i])

        names.append(v['name'][i])
        kind.append(k)
        if k == "tree_chunk":
            tree_chunk.append(255)
        else:
            tree_chunk.append(0)

x = np.sort(h_range)
h_range = np.expand_dims(h_range, axis=0)
h_range = np.repeat(h_range, 64, axis=0)
h_range = np.expand_dims(h_range, axis=0)
h_range = np.repeat(h_range, 64, axis=0)
h_range = np.array(h_range, dtype=np.float64)
h_range = torch.Tensor(h_range)

s_range = np.expand_dims(s_range, axis=0)
s_range = np.repeat(s_range, 64, axis=0)
s_range = np.expand_dims(s_range, axis=0)
s_range = np.repeat(s_range, 64, axis=0)
s_range = np.array(s_range, dtype=np.float64)
s_range = torch.Tensor(s_range)

v_range = np.expand_dims(v_range, axis=0)
v_range = np.repeat(v_range, 64, axis=0)
v_range = np.expand_dims(v_range, axis=0)
v_range = np.repeat(v_range, 64, axis=0)
v_range = np.array(v_range, dtype=np.float64)
v_range = torch.Tensor(v_range)



tree_chunk = np.array(tree_chunk, dtype=np.uint8)

keyboard_map = {'c': 'tree_chunk', 't': 'tree_leave', 'r': 'rock', 's': 'sand', 'd': 'dirt',
                'u': 'not_common', 'g':'grass','b':'black','w': 'water', 'k': 'sky', 'n':'snow',
                'm':'monster', 'a':'animal', 'h': 'coal', 'i': 'iron'}
key_selected = ''
add_done = False
while (True):
    # Display result image
    color_range = np.zeros([128, max_size, 3], dtype=np.uint8)
    tmp_time = time.time()

    if pasue:
        # action['place'] = 'furnace'
        # action['place'] = 'torch'

        # action['nearbyCraft'] = 'iron_pickaxe'
        # action['nearbyCraft'] = 'stone_pickaxe'
        action = model.predict(obs.copy())
        obs, reward, done, _ = env.step(action)
        t_R += reward
        # obs['inventory']
        # print(obs['inventory']['iron_ore'], end='\r')

        # record_p.append(obs['pov'])
    else:
        action = 0
    # if action['jump'] == 1:
    #     action['jump'] = 0
    image = obs['pov']

    # Convert to HSV format and color threshold
    X = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)


    tmp_mask = np.zeros([64, 64, 3], dtype=np.uint8)

    hsv_64 = np.array(hsv, dtype=np.float64)
    hsv_64 = torch.Tensor(hsv_64)
    pixels_maps = {}
    keys_mask = {}
    key_index = 0

    H_nei = np.zeros((64, 64, len(neighbor_pixels[0])), dtype=np.float64)
    S_nei = np.zeros((64, 64, len(neighbor_pixels[0])), dtype=np.float64)
    V_nei = np.zeros((64, 64, len(neighbor_pixels[0])), dtype=np.float64)

    for counter in range(len(neighbor_pixels[0])):
        new_hsv = shift_image(hsv_64, neighbor_pixels[0][counter],
                              neighbor_pixels[1][counter])
        # zero_index = np.where(new_hsv[:, :, 0] == 1500)
        # new_hsv[zero_index[0], zero_index[1], :] =
        new_hsv = np.abs(hsv_64 - new_hsv)
        index = np.where(new_hsv[:,:,0] >= 89.5)
        new_hsv[index[0], index[1], 0] = np.abs(179 - new_hsv[index[0], index[1], 0])
        H_nei[:, :, counter] = new_hsv[:, :, 0]
        S_nei[:, :, counter] = new_hsv[:, :, 1]
        V_nei[:, :, counter] = new_hsv[:, :, 1]


    # for i in range(64):
    #     for j in range(64):
    #         pixel_id = tuple((i, j))
    #         pixel_hsv = hsv_64[i, j, :]
    #         H_error = H_nei[i, j]
    #         H_error_min = np.min(H_error)
    #         S_error = S_nei[i, j]
    #         V_error = V_nei[i, j]
    #
    #         if H_error_min <= 1:
    #             key = None
    #             for ins in range(len(H_error)):
    #                 next_pixel = tuple((i - neighbor_pixels[1][ins],
    #                                     j - neighbor_pixels[0][ins]))
    #
    #                 if H_error[ins] <= 1 and S_error[ins] <= 1 and V_error[ins] <= 1:
    #                     # if next_pixel[0] < 0:
    #                     #     print(i, j, neighbor_pixels[0][ins])
    #                     if key is None:
    #                         if pixel_id in pixels_maps:
    #                             key = pixels_maps[pixel_id]
    #
    #                         elif next_pixel in pixels_maps:
    #                             key = pixels_maps[next_pixel]
    #                         else:
    #                             key = key_index
    #                             keys_mask[key] = [[], [], {}, [], []]
    #                             key_index += 1
    #                     # print(key)
    #                     pixels_maps[pixel_id] = key
    #                     pixels_maps[next_pixel] = key
    #                     if pixel_id not in keys_mask[key][2]:
    #                         keys_mask[key][2][pixel_id] = 0
    #                         keys_mask[key][0].append(pixel_id[0])
    #                         keys_mask[key][1].append(pixel_id[1])
    #
    #                     if next_pixel not in keys_mask[key][2]:
    #                         keys_mask[key][2][next_pixel] = 0
    #                         keys_mask[key][0].append(next_pixel[0])
    #                         keys_mask[key][1].append(next_pixel[1])

    # # print(len(keys_mask))
    # max = 0
    # k_m = 0
    # for k, v in keys_mask.items():
    #     hsv_64[v[0], v[1], 0] = np.mean(hsv[v[0], v[1], 0])
    #     hsv_64[v[0], v[1], 1] = np.mean(hsv[v[0], v[1], 1])
    #     hsv_64[v[0], v[1], 2] = np.mean(hsv[v[0], v[1], 2])
    #     # hsv[v[3], v[4], 0] = np.mean(hsv[v[3], v[4], 0])
    #     # hsv[v[3], v[4], 1] = np.mean(hsv[v[3], v[4], 1])
    #     # tmp_mask[v[3], v[4], 2] = np.mean(hsv[v[3], v[4], 2]) + 20
    # len(keys_mask)
    tmp_time = time.time()


    h = np.expand_dims(hsv_64[:, :, 0], axis=-1)
    s = np.expand_dims(hsv_64[:, :, 1], axis=-1)
    v = np.expand_dims(hsv_64[:, :, 2], axis=-1)
    h = torch.unsqueeze(hsv_64[:,:,0], -1)
    s = torch.unsqueeze(hsv_64[:,:,1], -1)
    v = torch.unsqueeze(hsv_64[:,:,2], -1)
    #
    # h = torch.Tensor(h)
    # s = torch.Tensor(s)
    # v = torch.Tensor(v)
    v.shape
    # H1_range.shape

    if len(dict_data['rock']['H']) != 0:

        h_error = torch.abs(h - h_range)
        index = torch.where(h_error >= 89.5)
        h_error[index[0], index[1], index[2]] = 179 - h_error[index[0], index[1], index[2]]


        tmp_33 = torch.sqrt(torch.pow(h_error, 2) + torch.pow((s/4 - s_range/4), 2)
                            + torch.pow((v/6 - v_range/6), 2))

        tmp_33 = torch.argmin(tmp_33, axis=2)
        # print(time.time() - tmp_time)
        x = tree_chunk[tmp_33]
        # tmp_image = np.expand_dims(tmp_image,axis=-1)
        x = np.expand_dims(x,axis=-1)
        x.shape
        index_color = 0
        for i in range(64):
            for j in range(64):
                pixel = hsv_64[i, j, :]
                index_x = tmp_33[i,j]
                kins = kind[index_x]
                # if kins not in kindss:
                #     kindss[kins] = index_color
                #     index_color += 1

                # tmp_mask[i, j, :] = np.array([hsv_r[names[index_x]]['H'],
                #                               hsv_r[names[index_x]]['S'],
                #                               hsv_r[names[index_x]]['v']], dtype=np.uint8)

                if kins in names_stack:
                    index_1 = np.where(np.array(names_stack) == kins)[0][0]
                else:
                    names_stack.append(kins)
                    index_1 = len(names_stack)
                color = pool_colors[index_1]
                tmp_mask[i, j, :] = color

                if (index_1 + 1) * 128 < max_size:
                    if color_range[0, index_1 * 128 +10, 0] == 0:
                        color_range[:, index_1 * 128: (index_1 + 1) * 128, :] = color
                        cv2.putText(color_range, kins,
                                    (index_1 * 128 + 20, 64),
                                    font,
                                    fontScale,
                                    fontColor,
                                    lineType)
        # print(time.time() - tmp_time)

    # tmp_33[29,22]
    if not lock and not pasue:
        a = 3
        if key_on_press in keyboard_map and key_on_release != 'l':
            key_selected = keyboard_map[key_on_press]
            print("new: key {0} - {1} - {2}".format(key_selected, [hsv_64[X_VALUE,Y_VALUE, 0],
                                                                   hsv_64[X_VALUE,Y_VALUE, 1],
                                                                   hsv_64[X_VALUE,Y_VALUE, 2]], [X_VALUE, Y_VALUE]))
            key_on_press = ''
            add_done = False

        if key_selected != '':
            if key_on_press in indexs:
                if int(key_on_press) == 1:
                    dict_data[key_selected]['H'].append(hsv_64[X_VALUE,Y_VALUE,0])
                    dict_data[key_selected]['S'].append(hsv_64[X_VALUE,Y_VALUE,1])
                    dict_data[key_selected]['V'].append(hsv_64[X_VALUE,Y_VALUE,2])

                    name = '{0}_{1}'.format(key_selected, len(dict_data[key_selected]['H']))

                    dict_data[key_selected]['name'].append(name)

                    print("ADDED : key {0} - {1} - {2}".format(key_selected, [H_VALUE, S_VALUE, V_VALUE],
                                                               [X_VALUE, Y_VALUE]))
                    with open('data_1.pkl', 'wb') as f:
                        pickle.dump(dict_data, f)


                if int(key_on_press) == 2:
                    print("STOP")
                    key_selected = ''
                    add_done = False
                if int(key_on_press) == 4:
                    h_del = dict_data[key_selected]['H'][-1]
                    s_del = dict_data[key_selected]['S'][-1]
                    v_del = dict_data[key_selected]['V'][-1]
                    name_del = dict_data[key_selected]['name'][-1]
                    dict_data[key_selected]['H'] = dict_data[key_selected]['H'][:-1]
                    dict_data[key_selected]['S'] = dict_data[key_selected]['S'][:-1]
                    dict_data[key_selected]['V'] = dict_data[key_selected]['V'][:-1]
                    dict_data[key_selected]['name'] = dict_data[key_selected]['name'][:-1]


                    print("DEL : key {0} - {1}".format(key_selected, [h_del, s_del, v_del]))
                    # key_selected = ''
                    add_done = False
                    with open('data_1.pkl', 'wb') as f:
                        pickle.dump(dict_data, f)

                h_range = []
                s_range = []
                v_range = []

                names = []
                kind = []
                for k, v in dict_data.items():
                    for i in range(len(v['H'])):
                        h_range.append(v['H'][i])
                        s_range.append(v['S'][i])
                        v_range.append(v['V'][i])

                        names.append(v['name'][i])
                        kind.append(k)
                print(len(h_range))

                h_range = np.expand_dims(h_range, axis=0)
                h_range = np.repeat(h_range, 64, axis=0)
                h_range = np.expand_dims(h_range, axis=0)
                h_range = np.repeat(h_range, 64, axis=0)
                h_range = np.array(h_range, dtype=np.float64)
                h_range = torch.Tensor(h_range)

                s_range = np.expand_dims(s_range, axis=0)
                s_range = np.repeat(s_range, 64, axis=0)
                s_range = np.expand_dims(s_range, axis=0)
                s_range = np.repeat(s_range, 64, axis=0)
                s_range = np.array(s_range, dtype=np.float64)
                s_range = torch.Tensor(s_range)

                v_range = np.expand_dims(v_range, axis=0)
                v_range = np.repeat(v_range, 64, axis=0)
                v_range = np.expand_dims(v_range, axis=0)
                v_range = np.repeat(v_range, 64, axis=0)
                v_range = np.array(v_range, dtype=np.float64)
                v_range = torch.Tensor(v_range)

                add_done = True
                key_on_press = ''
        #image = cv2.imread("./block_image/Mega_Spruce_Tree.png")

    # print(np.count_nonzero(names == 32)/4096, end='\r')
    # mask = 255 - mask
    # hsv[:,:,2] = 255
    # X = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)



    # X = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

    # dst = cv2.addWeighted(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), 0.7,
    #                       cv2.cvtColor(tmp_mask, cv2.COLOR_HSV2RGB), 0.7, 0)
    #hsv = np.array(hsv_64,dtype=np.uint8)
    # hsv[48,19,0] = hsv[48,18,0]
    # hsv[48,19,1] = hsv[48,18,1]
    #hsv[48,19,2]=  hsv[48,18,2]

    tmp_hsv = np.array(hsv)
    #tmp_hsv[X_VALUE,Y_VALUE,0] = 50

    tmp_hsv[X_VALUE,Y_VALUE,1] = 255

    tmp_hsv[X_VALUE,Y_VALUE,2] = 230
    #tmp_hsv[:,:,2] = 255

    # index = np.where((hsv[:, :, 0] >=20) |( hsv[:, :, 0] <= 16))
    #hsv[index[0], index[1], :] = [0,0, 255]
    # tmp_hsv[:,:,0] = 179
    # tmp_hsv[:,:,1] = 2
    # tmp_hsv[:,:,2] = tmp_hsv[:,:,2] + 50

    tmp_image = cv2.cvtColor(tmp_hsv, cv2.COLOR_HSV2BGR)
    # tmp_image = cv2.cvtColor(tmp_image, cv2.COLOR_BGR2GRAY)

    # tmp_image[0,1]
    # tmp_image[X_VALUE,Y_VALUE,:] = [100,100,100]
    tmp_mask[X_VALUE,Y_VALUE,2] = 230

    cv2.imshow('image',tmp_image)
    cv2.resizeWindow('image', 950, 950)
    cv2.setMouseCallback('image', Life2CodingRGB)

    cv2.imshow('mask', cv2.cvtColor(tmp_mask, cv2.COLOR_HSV2RGB))
    cv2.resizeWindow('mask', 950, 950)
    cv2.setMouseCallback('mask', Life2CodingRGB)


    cv2.imshow('color_range', cv2.cvtColor(color_range, cv2.COLOR_HSV2RGB))
    # cv2.resizeWindow('mask', 800, 800)

    # for c in cnts:
    #     x, y, w, h = cv2.boundingRect(c)
    #
    #     print(cv2.contourArea(c))
    # print("---")
    # image = obs['pov']
    # cv2.imshow('image', cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # print(time.time() - tmp_time)

    if cv2.waitKey(10) & 0xFF == ord('o'):
        break
    # time.sleep(0.1)

key_list.stop()
mouse_list.stop()

cv2.destroyAllWindows()
hsv[16,60]
# with open('data_1.pkl', 'wb') as f:
#     pickle.dump(dict_data, f)


# pixels_maps = {}
# keys_mask = {}
# key_index = 0
# hsv_64 = np.array(hsv, dtype=np.float64)
# for i in range(64):
#     for j in range(64):
#
#         pixel_id = tuple((i, j))
#         pixel_hsv = hsv_64[i, j, :]
#
#         neigh_bor = [neighbor_pixels[0] + i, neighbor_pixels[1] + j]
#         index = np.where(((neigh_bor[0] <= 63) & (neigh_bor[0] > 0)))
#         neigh_bor[0] = neigh_bor[0][index]
#         neigh_bor[1] = neigh_bor[1][index]
#         index = np.where(((neigh_bor[1] <= 63) & (neigh_bor[1] > 0)))
#         neigh_bor[0] = neigh_bor[0][index]
#         neigh_bor[1] = neigh_bor[1][index]
#
#         neigh_hs = hsv_64[neigh_bor[0], neigh_bor[1], : 2]
#
#         HS_error = np.sum(np.abs(neigh_hs - pixel_hsv[:2]), axis=1) / 2
#         HS_error_min = np.min(HS_error)
#         if HS_error_min <= 25:
#             for ins in range(len(HS_error)):
#                 if np.abs(HS_error[ins] - HS_error_min) <= 5:
#                     next_pixel = tuple((neigh_bor[0][ins], neigh_bor[1][ins]))
#                     if next_pixel in pixels_maps:
#                         key = pixels_maps[next_pixel]
#                     else:
#                         key = key_index
#                         keys_mask[key] = [[], [], {}]
#                         key_index += 1
#                     pixels_maps[pixel_id] = key
#                     pixels_maps[next_pixel] = key
#                     if pixel_id not in keys_mask[key][2]:
#                         keys_mask[key][2][pixel_id] = 0
#                         keys_mask[key][0].append(pixel_id[0])
#                         keys_mask[key][1].append(pixel_id[1])
#                     if next_pixel not in keys_mask[key][2]:
#                         keys_mask[key][2][next_pixel] = 0
#                         keys_mask[key][0].append(next_pixel[0])
#                         keys_mask[key][1].append(next_pixel[1])
# keys_mask[0]
# for k,v in keys_mask.items():
#     print(len(v[0]))
#     # tmp_mask[v[0], v[1],0] = np.mean(hsv[v[0], v[1],0])
#     # tmp_mask[v[0], v[1],1] = np.mean(hsv[v[0], v[1],1])
#     # tmp_mask[v[0], v[1],2] = np.mean(hsv[v[0], v[1],2])
a = 3
a = 3
# dict_data['dirt']
# dict_data['tree_chunk']

# dict_data = {}
# for k,v in keyboard_map.items():
#     kind = v
#     if kind not in dict_data:
#         dict_data[kind] = {'H': [], 'S': [], 'V': [],
#                            'H_1': [], 'S_1': [],
#                            'H_2': [], 'S_2': [],
#                            'name': []}
# with open('data_1.pkl', 'wb') as f:
#     pickle.dump(dict_data, f)
# dict_data[kind]['H'].append(v['H'])
# dict_data[kind]['S'].append(v['S'])
# dict_data[kind]['V'].append(v['v'])
# name = '{0}_{1}'.format(kind, len(dict_data[kind]['H']))
# dict_data[kind]['name'].append(name)
#


# with open('data.pkl', 'rb') as f:
#     output = pickle.load(f)
#
# h_range = []
# s_range = []
# v_range = []
# names = []
# kind = []
# for k,v in output.items():
#     for i in range(len(v['H'])):
#         h_range.append(v['H'][i])
#         s_range.append(v['S'][i])
#         v_range.append(v['V'][i])
#         names.append(v['name'][i])
#         kind.append(k)
# h_range = np.expand_dims(h_range,axis=0)
# h_range = np.repeat(h_range, 64, axis=0)
# h_range = np.expand_dims(h_range,axis=0)
# h_range = np.repeat(h_range, 64, axis=0)
#
# s_range = np.expand_dims(s_range,axis=0)
# s_range = np.repeat(s_range, 64, axis=0)
# s_range = np.expand_dims(s_range,axis=0)
# s_range = np.repeat(s_range, 64, axis=0)
# h_range = np.array(h_range, dtype=np.float64)
# s_range = np.array(s_range, dtype=np.float64)
#
# v_range = np.expand_dims(v_range,axis=0)
# v_range = np.repeat(v_range, 64, axis=0)
# v_range = np.expand_dims(v_range,axis=0)
# v_range = np.repeat(v_range, 64, axis=0)
# v_range = np.array(v_range, dtype=np.float64)

# key_del = 'tree_chunk'
# item_del = 'tree_chunk_4'
# tmp_key = {'H': [], 'S': [], 'V': [],
#                            'D': [],
#                            'name': []}
# v = dict_data[key_del]
# for i in range(len(dict_data[key_del]['H'])):
#     if v['name'][i] != item_del:
#         tmp_key['H'].append(v['H'][i])
#         tmp_key['S'].append(v['S'][i])
#         tmp_key['V'].append(v['V'][i])
#         tmp_key['D'].append(v['D'][i])
#         name = '{0}_{1}'.format(kind, len(tmp_key['H']))
#         tmp_key['name'].append(name)
#
# dict_data[key_del] = tmp_key

In [26]:
import minerl
import gym
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np, cv2
from pynput.keyboard import Key, Listener
from pynput.mouse import Button
from pynput import mouse
from threading import Thread
import pickle
import time
import random
import imutils
from pairing import pair, depair
import torch
torch.cuda.set_device(0)
# class PixelConnect():
#     def __init__(self):
#         super.__init__()
#
# class CompareNeighbor():
#     def __init__(self, max_neighbor_pixel=1, image_size=64):
#         super.__init__()
#         x = np.linspace(-max_neighbor_pixel, max_neighbor_pixel, max_neighbor_pixel * 2 + 1)
#         self.neighbor_pixels =  [[], []]
#         for i in range(len(x)):
#             for j in range(len(y)):
#                 if x[i] != 0 and x[j] != 0:
#                     self.neighbor_pixels[0].append(x[i])
#                     self.neighbor_pixels[1].append(x[j])
#
#
#         x = np.arange(1, image_size, 2)
#         self.pixels_map = {}
#         for i in range(len(x)):
#             for j in range(len(x)):
#                 self.pixels_map[tuple((x[i], x[j]))] = PixelConnect()
#
#     def make_pixel_connect(self, image):
#         hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
#         for pixel in self.pixels_map:
#

a = 3

# obs = env.reset()


# env = gym.make('MineRLObtainDiamond-v0')
# obs = env.reset()
# record_p = []

# hsv_bond = {}
#
# key_list.stop()
# mouse_list.stop()




def nothing(x):
    pass


def trafer_hsv(dict):
    cv2.setTrackbarPos('HMin', 'image', dict['hMin'])
    cv2.setTrackbarPos('SMin', 'image', dict['sMin'])
    if 'vMin' in dict:
        cv2.setTrackbarPos('VMin', 'image', dict['vMin'])
    else:
        cv2.setTrackbarPos('VMin', 'image', 0)

    cv2.setTrackbarPos('HMax', 'image', dict['hMax'])
    cv2.setTrackbarPos('SMax', 'image', dict['sMax'])

    if 'vMax' in dict:
        cv2.setTrackbarPos('VMax', 'image', dict['vMax'])
    else:
        cv2.setTrackbarPos('VMax', 'image', 255)


def on_move(x, y):
    global attack_b


def on_click(x, y, button, pressed):
    global attack_b
    global action
    # print('{0} at {1}'.format(
    #     button if pressed else 'Released',
    #     (x, y)))
    if pressed:
        try:
            if button == Button.left:
                action['attack'] = 1
        except AttributeError:
            pass
    else:
        action['attack'] = 0
    global key_on_release
    if key_on_release == 'esc':
        return False


def on_scroll(x, y, dx, dy):
    global attack_b


def on_press(key):
    global key_on_press
    global action
    try:
        key_on_press = key.char
        if key_on_press == 'w' and action['back'] != 1:
            action['forward'] = 1

        if key_on_press == 's' and action['forward'] != 1:
            action['back'] = 1

        if key_on_press == 'a' and action['right'] != 1:
            action['left'] = 1
        if key_on_press == 'd' and action['left'] != 1:
            action['right'] = 1

        if key_on_press == 'q' and action['camera'][1] == 0:
            action['camera'][1] = -15

        if key_on_press == 'e' and action['camera'][1] == 0:
            action['camera'][1] = 15

        if key_on_press == 'r' and action['camera'][0] == 0:
            action['camera'][0] = -15
        if key_on_press == 'f' and action['camera'][0] == 0:
            action['camera'][0] = 15


    except AttributeError:
        # do something when a certain key is pressed, using key, not key.char
        pass
    if key == Key.space:
        action['jump'] = 1


def on_release(key):
    global key_on_release
    global jump
    global action
    global pasue
    global lock
    try:
        key_on_release = key.char
        if key_on_release == 'w':
            action['forward'] = 0
        if key_on_release == 's':
            action['back'] = 0
        if key_on_release == 'a':
            action['left'] = 0
        if key_on_release == 'd':
            action['right'] = 0

        if key_on_release == 'q':
            action['camera'][1] = 0
        if key_on_release == 'e':
            action['camera'][1] = 0
        if key_on_release == 'r':
            action['camera'][0] = 0
        if key_on_release == 'f':
            action['camera'][0] = 0
        if key_on_release == 'p':
            pasue = not pasue
            print("pause ", pasue)
        if key_on_release == 'l':
            lock = not lock
            print(lock)
        # if key.char == 'o':
        #     return False
    except AttributeError:
        # do something when a certain key is pressed, using key, not key.char
        pass

    if key == Key.esc:
        # Stop listener
        print("Stop lister ")
        key_on_release = 'esc'
        return False
    if key == Key.space:
        action['jump'] = 0

def shift_image(X, dx, dy):
    X = np.array(X, dtype=np.float64)
    X = np.roll(X, dy, axis=0)
    X = np.roll(X, dx, axis=1)
    if dy>0:
        X[:dy, :] = 1500
    elif dy<0:
        X[dy:, :] = 1500
    if dx>0:
        X[:, :dx] = 1500
    elif dx<0:
        X[:, dx:] = 1500
    return X

def Life2CodingRGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  # checks mouse moves
        global hsv
        #image_2 = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        colorsBGR = hsv[y, x]
        # colorsRGB=cv2.cvtColor(colorsBGR,cv2.COLOR_BGR2HSV)
        global tmp_33
        global names
        global dict_data
        global h_range
        global s_range
        global v_range
        global X_VALUE
        global Y_VALUE
        X_VALUE = y
        Y_VALUE = x
        H_VALUE = colorsBGR[0]
        S_VALUE = colorsBGR[1]
        V_VALUE = colorsBGR[2]
        index = tmp_33[y,x]

        print("RGB Value at ({},{}):{} {} {}".format(x,y,colorsBGR, names[index],
                                                     [h_range[y,x,index],
                                                      s_range[y,x,index],
                                                      v_range[y,x,index]]), end='\r')

def Life2CodingRGB2(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  # checks mouse moves
        image_2  =cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        colorsBGR = image_2[y, x]
        # colorsRGB=cv2.cvtColor(colorsBGR,cv2.COLOR_BGR2HSV)
        global tmp_33
        global names
        print("RGB Value at ({},{}):{} {}".format(x,y,colorsBGR, names[tmp_33[y,x]]), end='\r')


cv2.namedWindow('mask', cv2.WINDOW_KEEPRATIO)
cv2.resizeWindow('mask', 1000, 1000)
cv2.moveWindow('mask', 1100, 30)
cv2.namedWindow('image', cv2.WINDOW_KEEPRATIO)
cv2.resizeWindow('image', 1000, 1000)
cv2.moveWindow('image', -30, 30)



cv2.namedWindow('color_range')
cv2.moveWindow('color_range', 40, 1000)

# cv2.createTrackbar('HMin', 'image', 0, 179, nothing)
# cv2.createTrackbar('SMin', 'image', 0, 255, nothing)
# cv2.createTrackbar('HMax', 'image', 0, 179, nothing)
# cv2.createTrackbar('SMax', 'image', 0, 255, nothing)
# cv2.createTrackbar('VMin', 'image', 0, 255, nothing)
# cv2.createTrackbar('VMax', 'image', 0, 255, nothing)
# # Set default value for Max HSV trackbars
# cv2.setTrackbarPos('HMax', 'image', 179)
# cv2.setTrackbarPos('SMax', 'image', 255)
# cv2.setTrackbarPos('VMax', 'image', 255)

# Initialize HSV min/max values
hMin = sMin = vMin = hMax = sMax = vMax = 0
phMin = psMin = pvMin = phMax = psMax = pvMax = 0

index = 1
# action = env.action_space.noop()
key_list = Listener(on_press=on_press, on_release=on_release)
mouse_list = mouse.Listener(on_move=on_move, on_click=on_click, on_scroll=on_scroll)
attack_b = False
key_list.start()
mouse_list.start()

key_on_press = ''
key_on_release = ''
pasue = False
lock = True
levels = [-1, -1, -1]
current_level = -1
indexs = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

pool_size = 20
pool_colors = np.zeros([pool_size,3])
np.random.seed(10)
x = list(np.arange(0, 179, 179/pool_size))
pool_colors[:,0] =np.random.permutation(x)
pool_colors[:,1] = np.random.choice(range(50, 200), size=pool_size)
pool_colors[:,2] = np.random.choice(range(100, 200), size=pool_size)

names_stack = []
t_R = 0
font = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (400, 400)
fontScale = 0.6
fontColor = (0, 0, 255)
lineType = 1

max_size = 128 * 15

x = np.linspace(-1, 1, 3, dtype=np.int64)
# x = [-2, -1, 1, 2]
# neighbor_pixels = [[-1, 0, 1, 0, -2, 2, 2, -2], [0, 1, 0, -1, -2, -2, 2, 2]]
neighbor_pixels = [[], []]
for i in range(len(x)):
    for j in range(len(x)):
        if not (x[i] == 0 and x[j] == 0):
            neighbor_pixels[0].append(x[i])
            neighbor_pixels[1].append(x[j])
neighbor_pixels = np.array(neighbor_pixels)
H_VALUE = 0
S_VALUE = 0
V_VALUE = 0
X_VALUE = 0
Y_VALUE = 0
dict_data = {}
with open('data_1.pkl', 'rb') as f:
    dict_data = pickle.load(f)

h_range = []
s_range = []
v_range = []

names = []
kind = []
tree_chunk = []
for k, v in dict_data.items():
    for i in range(len(v['H'])):
        h_range.append(v['H'][i])
        s_range.append(v['S'][i])
        v_range.append(v['V'][i])

        names.append(v['name'][i])
        kind.append(k)
        if k == "tree_chunk":
            tree_chunk.append(255)
        else:
            tree_chunk.append(0)

x = np.sort(h_range)
h_range = np.expand_dims(h_range, axis=0)
h_range = np.repeat(h_range, 64, axis=0)
h_range = np.expand_dims(h_range, axis=0)
h_range = np.repeat(h_range, 64, axis=0)
h_range = np.array(h_range, dtype=np.float64)
h_range = torch.Tensor(h_range)

s_range = np.expand_dims(s_range, axis=0)
s_range = np.repeat(s_range, 64, axis=0)
s_range = np.expand_dims(s_range, axis=0)
s_range = np.repeat(s_range, 64, axis=0)
s_range = np.array(s_range, dtype=np.float64)
s_range = torch.Tensor(s_range)

v_range = np.expand_dims(v_range, axis=0)
v_range = np.repeat(v_range, 64, axis=0)
v_range = np.expand_dims(v_range, axis=0)
v_range = np.repeat(v_range, 64, axis=0)
v_range = np.array(v_range, dtype=np.float64)
v_range = torch.Tensor(v_range)



tree_chunk = np.array(tree_chunk, dtype=np.uint8)

keyboard_map = {'c': 'tree_chunk', 't': 'tree_leave', 'r': 'rock', 's': 'sand', 'd': 'dirt',
                'u': 'not_common', 'g':'grass','b':'black','w': 'water', 'k': 'sky', 'n':'snow',
                'm':'monster', 'a':'animal', 'h': 'coal', 'i': 'iron'}
key_selected = ''
add_done = False
while (True):
    # Display result image
    color_range = np.zeros([128, max_size, 3], dtype=np.uint8)
    tmp_time = time.time()

    if pasue:
        # action['place'] = 'furnace'
        # action['place'] = 'torch'

        # action['nearbyCraft'] = 'iron_pickaxe'
        # action['nearbyCraft'] = 'stone_pickaxe'
        action = model.predict(obs.copy())
        obs, reward, done, _ = env.step(action)
        t_R += reward
        # obs['inventory']
        # print(obs['inventory']['iron_ore'], end='\r')

        # record_p.append(obs['pov'])
    else:
        action = 0
    # if action['jump'] == 1:
    #     action['jump'] = 0
    image = obs['pov']

    # Convert to HSV format and color threshold
    X = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)


    tmp_mask = np.zeros([64, 64, 3], dtype=np.uint8)

    hsv_64 = np.array(hsv, dtype=np.float64)
    hsv_64 = torch.Tensor(hsv_64)
    pixels_maps = {}
    keys_mask = {}
    key_index = 0

    H_nei = np.zeros((64, 64, len(neighbor_pixels[0])), dtype=np.float64)
    S_nei = np.zeros((64, 64, len(neighbor_pixels[0])), dtype=np.float64)
    V_nei = np.zeros((64, 64, len(neighbor_pixels[0])), dtype=np.float64)

    for counter in range(len(neighbor_pixels[0])):
        new_hsv = shift_image(hsv_64, neighbor_pixels[0][counter],
                              neighbor_pixels[1][counter])
        # zero_index = np.where(new_hsv[:, :, 0] == 1500)
        # new_hsv[zero_index[0], zero_index[1], :] =
        new_hsv = np.abs(hsv_64 - new_hsv)
        index = np.where(new_hsv[:,:,0] >= 89.5)
        new_hsv[index[0], index[1], 0] = np.abs(179 - new_hsv[index[0], index[1], 0])
        H_nei[:, :, counter] = new_hsv[:, :, 0]
        S_nei[:, :, counter] = new_hsv[:, :, 1]
        V_nei[:, :, counter] = new_hsv[:, :, 1]


    # for i in range(64):
    #     for j in range(64):
    #         pixel_id = tuple((i, j))
    #         pixel_hsv = hsv_64[i, j, :]
    #         H_error = H_nei[i, j]
    #         H_error_min = np.min(H_error)
    #         S_error = S_nei[i, j]
    #         V_error = V_nei[i, j]
    #
    #         if H_error_min <= 1:
    #             key = None
    #             for ins in range(len(H_error)):
    #                 next_pixel = tuple((i - neighbor_pixels[1][ins],
    #                                     j - neighbor_pixels[0][ins]))
    #
    #                 if H_error[ins] <= 1 and S_error[ins] <= 1 and V_error[ins] <= 1:
    #                     # if next_pixel[0] < 0:
    #                     #     print(i, j, neighbor_pixels[0][ins])
    #                     if key is None:
    #                         if pixel_id in pixels_maps:
    #                             key = pixels_maps[pixel_id]
    #
    #                         elif next_pixel in pixels_maps:
    #                             key = pixels_maps[next_pixel]
    #                         else:
    #                             key = key_index
    #                             keys_mask[key] = [[], [], {}, [], []]
    #                             key_index += 1
    #                     # print(key)
    #                     pixels_maps[pixel_id] = key
    #                     pixels_maps[next_pixel] = key
    #                     if pixel_id not in keys_mask[key][2]:
    #                         keys_mask[key][2][pixel_id] = 0
    #                         keys_mask[key][0].append(pixel_id[0])
    #                         keys_mask[key][1].append(pixel_id[1])
    #
    #                     if next_pixel not in keys_mask[key][2]:
    #                         keys_mask[key][2][next_pixel] = 0
    #                         keys_mask[key][0].append(next_pixel[0])
    #                         keys_mask[key][1].append(next_pixel[1])

    # # print(len(keys_mask))
    # max = 0
    # k_m = 0
    # for k, v in keys_mask.items():
    #     hsv_64[v[0], v[1], 0] = np.mean(hsv[v[0], v[1], 0])
    #     hsv_64[v[0], v[1], 1] = np.mean(hsv[v[0], v[1], 1])
    #     hsv_64[v[0], v[1], 2] = np.mean(hsv[v[0], v[1], 2])
    #     # hsv[v[3], v[4], 0] = np.mean(hsv[v[3], v[4], 0])
    #     # hsv[v[3], v[4], 1] = np.mean(hsv[v[3], v[4], 1])
    #     # tmp_mask[v[3], v[4], 2] = np.mean(hsv[v[3], v[4], 2]) + 20
    # len(keys_mask)
    tmp_time = time.time()


    h = np.expand_dims(hsv_64[:, :, 0], axis=-1)
    s = np.expand_dims(hsv_64[:, :, 1], axis=-1)
    v = np.expand_dims(hsv_64[:, :, 2], axis=-1)
    h = torch.unsqueeze(hsv_64[:,:,0], -1)
    s = torch.unsqueeze(hsv_64[:,:,1], -1)
    v = torch.unsqueeze(hsv_64[:,:,2], -1)
    #
    # h = torch.Tensor(h)
    # s = torch.Tensor(s)
    # v = torch.Tensor(v)
    v.shape
    # H1_range.shape

    if len(dict_data['rock']['H']) != 0:

        h_error = torch.abs(h - h_range)
        index = torch.where(h_error >= 89.5)
        h_error[index[0], index[1], index[2]] = 179 - h_error[index[0], index[1], index[2]]


        tmp_33 = torch.sqrt(torch.pow(h_error, 2) + torch.pow((s/4 - s_range/4), 2)
                            + torch.pow((v/6 - v_range/6), 2))

        tmp_33 = torch.argmin(tmp_33, axis=2)
        # print(time.time() - tmp_time)
        x = tree_chunk[tmp_33]
        # tmp_image = np.expand_dims(tmp_image,axis=-1)
        x = np.expand_dims(x,axis=-1)
        x.shape
        index_color = 0
        for i in range(64):
            for j in range(64):
                pixel = hsv_64[i, j, :]
                index_x = tmp_33[i,j]
                kins = kind[index_x]
                # if kins not in kindss:
                #     kindss[kins] = index_color
                #     index_color += 1

                # tmp_mask[i, j, :] = np.array([hsv_r[names[index_x]]['H'],
                #                               hsv_r[names[index_x]]['S'],
                #                               hsv_r[names[index_x]]['v']], dtype=np.uint8)

                if kins in names_stack:
                    index_1 = np.where(np.array(names_stack) == kins)[0][0]
                else:
                    names_stack.append(kins)
                    index_1 = len(names_stack)
                color = pool_colors[index_1]
                tmp_mask[i, j, :] = color

                if (index_1 + 1) * 128 < max_size:
                    if color_range[0, index_1 * 128 +10, 0] == 0:
                        color_range[:, index_1 * 128: (index_1 + 1) * 128, :] = color
                        cv2.putText(color_range, kins,
                                    (index_1 * 128 + 20, 64),
                                    font,
                                    fontScale,
                                    fontColor,
                                    lineType)
        # print(time.time() - tmp_time)

    # tmp_33[29,22]
    if not lock and not pasue:
        a = 3
        if key_on_press in keyboard_map and key_on_release != 'l':
            key_selected = keyboard_map[key_on_press]
            print("new: key {0} - {1} - {2}".format(key_selected, [hsv_64[X_VALUE,Y_VALUE, 0],
                                                                   hsv_64[X_VALUE,Y_VALUE, 1],
                                                                   hsv_64[X_VALUE,Y_VALUE, 2]], [X_VALUE, Y_VALUE]))
            key_on_press = ''
            add_done = False

        if key_selected != '':
            if key_on_press in indexs:
                if int(key_on_press) == 1:
                    dict_data[key_selected]['H'].append(hsv_64[X_VALUE,Y_VALUE,0])
                    dict_data[key_selected]['S'].append(hsv_64[X_VALUE,Y_VALUE,1])
                    dict_data[key_selected]['V'].append(hsv_64[X_VALUE,Y_VALUE,2])

                    name = '{0}_{1}'.format(key_selected, len(dict_data[key_selected]['H']))

                    dict_data[key_selected]['name'].append(name)

                    print("ADDED : key {0} - {1} - {2}".format(key_selected, [H_VALUE, S_VALUE, V_VALUE],
                                                               [X_VALUE, Y_VALUE]))
                    with open('data_1.pkl', 'wb') as f:
                        pickle.dump(dict_data, f)


                if int(key_on_press) == 2:
                    print("STOP")
                    key_selected = ''
                    add_done = False
                if int(key_on_press) == 4:
                    h_del = dict_data[key_selected]['H'][-1]
                    s_del = dict_data[key_selected]['S'][-1]
                    v_del = dict_data[key_selected]['V'][-1]
                    name_del = dict_data[key_selected]['name'][-1]
                    dict_data[key_selected]['H'] = dict_data[key_selected]['H'][:-1]
                    dict_data[key_selected]['S'] = dict_data[key_selected]['S'][:-1]
                    dict_data[key_selected]['V'] = dict_data[key_selected]['V'][:-1]
                    dict_data[key_selected]['name'] = dict_data[key_selected]['name'][:-1]


                    print("DEL : key {0} - {1}".format(key_selected, [h_del, s_del, v_del]))
                    # key_selected = ''
                    add_done = False
                    with open('data_1.pkl', 'wb') as f:
                        pickle.dump(dict_data, f)

                h_range = []
                s_range = []
                v_range = []

                names = []
                kind = []
                for k, v in dict_data.items():
                    for i in range(len(v['H'])):
                        h_range.append(v['H'][i])
                        s_range.append(v['S'][i])
                        v_range.append(v['V'][i])

                        names.append(v['name'][i])
                        kind.append(k)
                print(len(h_range))

                h_range = np.expand_dims(h_range, axis=0)
                h_range = np.repeat(h_range, 64, axis=0)
                h_range = np.expand_dims(h_range, axis=0)
                h_range = np.repeat(h_range, 64, axis=0)
                h_range = np.array(h_range, dtype=np.float64)
                h_range = torch.Tensor(h_range)

                s_range = np.expand_dims(s_range, axis=0)
                s_range = np.repeat(s_range, 64, axis=0)
                s_range = np.expand_dims(s_range, axis=0)
                s_range = np.repeat(s_range, 64, axis=0)
                s_range = np.array(s_range, dtype=np.float64)
                s_range = torch.Tensor(s_range)

                v_range = np.expand_dims(v_range, axis=0)
                v_range = np.repeat(v_range, 64, axis=0)
                v_range = np.expand_dims(v_range, axis=0)
                v_range = np.repeat(v_range, 64, axis=0)
                v_range = np.array(v_range, dtype=np.float64)
                v_range = torch.Tensor(v_range)

                add_done = True
                key_on_press = ''
        #image = cv2.imread("./block_image/Mega_Spruce_Tree.png")

    # print(np.count_nonzero(names == 32)/4096, end='\r')
    # mask = 255 - mask
    # hsv[:,:,2] = 255
    # X = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)



    # X = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

    # dst = cv2.addWeighted(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), 0.7,
    #                       cv2.cvtColor(tmp_mask, cv2.COLOR_HSV2RGB), 0.7, 0)
    #hsv = np.array(hsv_64,dtype=np.uint8)
    # hsv[48,19,0] = hsv[48,18,0]
    # hsv[48,19,1] = hsv[48,18,1]
    #hsv[48,19,2]=  hsv[48,18,2]

    tmp_hsv = np.array(hsv)
    #tmp_hsv[X_VALUE,Y_VALUE,0] = 50

    tmp_hsv[X_VALUE,Y_VALUE,1] = 255

    tmp_hsv[X_VALUE,Y_VALUE,2] = 230
    #tmp_hsv[:,:,2] = 255

    # index = np.where((hsv[:, :, 0] >=20) |( hsv[:, :, 0] <= 16))
    #hsv[index[0], index[1], :] = [0,0, 255]
    # tmp_hsv[:,:,0] = 179
    # tmp_hsv[:,:,1] = 2
    # tmp_hsv[:,:,2] = tmp_hsv[:,:,2] + 50

    tmp_image = cv2.cvtColor(tmp_hsv, cv2.COLOR_HSV2BGR)
    # tmp_image = cv2.cvtColor(tmp_image, cv2.COLOR_BGR2GRAY)

    # tmp_image[0,1]
    # tmp_image[X_VALUE,Y_VALUE,:] = [100,100,100]
    tmp_mask[X_VALUE,Y_VALUE,2] = 230

    cv2.imshow('image',tmp_image)
    cv2.resizeWindow('image', 950, 950)
    cv2.setMouseCallback('image', Life2CodingRGB)

    cv2.imshow('mask', cv2.cvtColor(tmp_mask, cv2.COLOR_HSV2RGB))
    cv2.resizeWindow('mask', 950, 950)
    cv2.setMouseCallback('mask', Life2CodingRGB)


    cv2.imshow('color_range', cv2.cvtColor(color_range, cv2.COLOR_HSV2RGB))
    # cv2.resizeWindow('mask', 800, 800)

    # for c in cnts:
    #     x, y, w, h = cv2.boundingRect(c)
    #
    #     print(cv2.contourArea(c))
    # print("---")
    # image = obs['pov']
    # cv2.imshow('image', cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # print(time.time() - tmp_time)

    if cv2.waitKey(10) & 0xFF == ord('o'):
        break
    # time.sleep(0.1)

key_list.stop()
mouse_list.stop()

cv2.destroyAllWindows()
hsv[16,60]
# with open('data_1.pkl', 'wb') as f:
#     pickle.dump(dict_data, f)


# pixels_maps = {}
# keys_mask = {}
# key_index = 0
# hsv_64 = np.array(hsv, dtype=np.float64)
# for i in range(64):
#     for j in range(64):
#
#         pixel_id = tuple((i, j))
#         pixel_hsv = hsv_64[i, j, :]
#
#         neigh_bor = [neighbor_pixels[0] + i, neighbor_pixels[1] + j]
#         index = np.where(((neigh_bor[0] <= 63) & (neigh_bor[0] > 0)))
#         neigh_bor[0] = neigh_bor[0][index]
#         neigh_bor[1] = neigh_bor[1][index]
#         index = np.where(((neigh_bor[1] <= 63) & (neigh_bor[1] > 0)))
#         neigh_bor[0] = neigh_bor[0][index]
#         neigh_bor[1] = neigh_bor[1][index]
#
#         neigh_hs = hsv_64[neigh_bor[0], neigh_bor[1], : 2]
#
#         HS_error = np.sum(np.abs(neigh_hs - pixel_hsv[:2]), axis=1) / 2
#         HS_error_min = np.min(HS_error)
#         if HS_error_min <= 25:
#             for ins in range(len(HS_error)):
#                 if np.abs(HS_error[ins] - HS_error_min) <= 5:
#                     next_pixel = tuple((neigh_bor[0][ins], neigh_bor[1][ins]))
#                     if next_pixel in pixels_maps:
#                         key = pixels_maps[next_pixel]
#                     else:
#                         key = key_index
#                         keys_mask[key] = [[], [], {}]
#                         key_index += 1
#                     pixels_maps[pixel_id] = key
#                     pixels_maps[next_pixel] = key
#                     if pixel_id not in keys_mask[key][2]:
#                         keys_mask[key][2][pixel_id] = 0
#                         keys_mask[key][0].append(pixel_id[0])
#                         keys_mask[key][1].append(pixel_id[1])
#                     if next_pixel not in keys_mask[key][2]:
#                         keys_mask[key][2][next_pixel] = 0
#                         keys_mask[key][0].append(next_pixel[0])
#                         keys_mask[key][1].append(next_pixel[1])
# keys_mask[0]
# for k,v in keys_mask.items():
#     print(len(v[0]))
#     # tmp_mask[v[0], v[1],0] = np.mean(hsv[v[0], v[1],0])
#     # tmp_mask[v[0], v[1],1] = np.mean(hsv[v[0], v[1],1])
#     # tmp_mask[v[0], v[1],2] = np.mean(hsv[v[0], v[1],2])
a = 3
a = 3
# dict_data['dirt']
# dict_data['tree_chunk']

# dict_data = {}
# for k,v in keyboard_map.items():
#     kind = v
#     if kind not in dict_data:
#         dict_data[kind] = {'H': [], 'S': [], 'V': [],
#                            'H_1': [], 'S_1': [],
#                            'H_2': [], 'S_2': [],
#                            'name': []}
# with open('data_1.pkl', 'wb') as f:
#     pickle.dump(dict_data, f)
# dict_data[kind]['H'].append(v['H'])
# dict_data[kind]['S'].append(v['S'])
# dict_data[kind]['V'].append(v['v'])
# name = '{0}_{1}'.format(kind, len(dict_data[kind]['H']))
# dict_data[kind]['name'].append(name)
#


# with open('data.pkl', 'rb') as f:
#     output = pickle.load(f)
#
# h_range = []
# s_range = []
# v_range = []
# names = []
# kind = []
# for k,v in output.items():
#     for i in range(len(v['H'])):
#         h_range.append(v['H'][i])
#         s_range.append(v['S'][i])
#         v_range.append(v['V'][i])
#         names.append(v['name'][i])
#         kind.append(k)
# h_range = np.expand_dims(h_range,axis=0)
# h_range = np.repeat(h_range, 64, axis=0)
# h_range = np.expand_dims(h_range,axis=0)
# h_range = np.repeat(h_range, 64, axis=0)
#
# s_range = np.expand_dims(s_range,axis=0)
# s_range = np.repeat(s_range, 64, axis=0)
# s_range = np.expand_dims(s_range,axis=0)
# s_range = np.repeat(s_range, 64, axis=0)
# h_range = np.array(h_range, dtype=np.float64)
# s_range = np.array(s_range, dtype=np.float64)
#
# v_range = np.expand_dims(v_range,axis=0)
# v_range = np.repeat(v_range, 64, axis=0)
# v_range = np.expand_dims(v_range,axis=0)
# v_range = np.repeat(v_range, 64, axis=0)
# v_range = np.array(v_range, dtype=np.float64)

# key_del = 'tree_chunk'
# item_del = 'tree_chunk_4'
# tmp_key = {'H': [], 'S': [], 'V': [],
#                            'D': [],
#                            'name': []}
# v = dict_data[key_del]
# for i in range(len(dict_data[key_del]['H'])):
#     if v['name'][i] != item_del:
#         tmp_key['H'].append(v['H'][i])
#         tmp_key['S'].append(v['S'][i])
#         tmp_key['V'].append(v['V'][i])
#         tmp_key['D'].append(v['D'][i])
#         name = '{0}_{1}'.format(kind, len(tmp_key['H']))
#         tmp_key['name'].append(name)
#
# dict_data[key_del] = tmp_key

In [27]:
key_list.stop()
mouse_list.stop()

In [28]:
obs

array([[[128,   0],
        [128,   0],
        [127,   0],
        ...,
        [162,   0],
        [162,   0],
        [118,   0]],

       [[120,   0],
        [126,   0],
        [128,   0],
        ...,
        [162,   0],
        [161,   0],
        [123,   0]],

       [[118,   0],
        [122,   0],
        [125,   0],
        ...,
        [164,   0],
        [161,   0],
        [124,   0]],

       ...,

       [[135,   0],
        [134,   0],
        [141,   0],
        ...,
        [133,   0],
        [136,   0],
        [127,   0]],

       [[124,   0],
        [122,   0],
        [122,   0],
        ...,
        [121,   0],
        [116,   0],
        [116,   0]],

       [[128,   0],
        [131,   0],
        [127,   0],
        ...,
        [120,   0],
        [125,   0],
        [126,   0]]], dtype=uint8)

In [29]:
cv2.destroyAllWindows()

In [30]:


import time
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common import env_checker

from torch.utils.tensorboard import SummaryWriter
import minerl  # it's important to import minerl after SB3, otherwise model.save doesn't work...
import cv2
import numpy as np
import pickle
import torch
try:
    wandb = None
    import wandb
except ImportError:
    pass

class PovOnlyObservation(gym.ObservationWrapper):

    def __init__(self, env):
        super().__init__(env)
        # 64 x 64 + 64 x64 + 18 [cpa;, cobblestone]
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(8210,), dtype=np.float64)
        self.observation_space = self.env.observation_space['pov']
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(4114,), dtype=np.float64)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(64, 64, 2), dtype=np.uint8)

        self.names_stack = []
        self.max_name = 120
        self.max_item = 50

        dict_data = {}
        with open('data_1.pkl', 'rb') as f:
            dict_data = pickle.load(f)



        h_range = []
        s_range = []
        v_range = []

        names = []
        kind = []
        counter = 0
        tree = 0
        for k, v in dict_data.items():
            for i in range(len(v['H'])):
                h_range.append(v['H'][i])
                s_range.append(v['S'][i])
                v_range.append(v['V'][i])

                names.append(v['name'][i])
                if k == "tree_chunk":
                    kind.append(255)
                else:
                    kind.append(0)

            # if k == tre
            # counter += 1

        h_range = np.expand_dims(h_range, axis=0)
        h_range = np.repeat(h_range, 64, axis=0)
        h_range = np.expand_dims(h_range, axis=0)
        h_range = np.repeat(h_range, 64, axis=0)
        h_range = np.array(h_range, dtype=np.float64)
        self.h_range = torch.Tensor(h_range)

        s_range = np.expand_dims(s_range, axis=0)
        s_range = np.repeat(s_range, 64, axis=0)
        s_range = np.expand_dims(s_range, axis=0)
        s_range = np.repeat(s_range, 64, axis=0)
        s_range = np.array(s_range, dtype=np.float64)
        self.s_range = torch.Tensor(s_range)


        v_range = np.expand_dims(v_range, axis=0)
        v_range = np.repeat(v_range, 64, axis=0)
        v_range = np.expand_dims(v_range, axis=0)
        v_range = np.repeat(v_range, 64, axis=0)
        v_range = np.array(v_range, dtype=np.float64)
        self.v_range = torch.Tensor(v_range)

        self.kind = np.array(kind, dtype=np.uint8)

    def observation(self, observation):
        image = observation['pov']
        hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        hsv_64 = np.array(hsv, dtype=np.float64)
        hsv_64 = torch.Tensor(hsv_64)

        h = torch.unsqueeze(hsv_64[:, :, 0], -1)
        s = torch.unsqueeze(hsv_64[:, :, 1], -1)
        v = torch.unsqueeze(hsv_64[:, :, 2], -1)

        h_error = torch.abs(h - self.h_range)
        index = torch.where(h_error >= 89.5)
        h_error[index[0], index[1], index[2]] = 179 - h_error[index[0], index[1], index[2]]

        d_error = torch.sqrt(torch.pow(h_error, 2) + torch.pow((s / 4 - self.s_range / 4), 2)
                             + torch.pow((v / 6 - self.v_range / 6), 2))

        d_error = torch.argmin(d_error, axis=2)
        tree_obs = self.kind[d_error]
        tree_obs = np.array(tree_obs, dtype=np.uint8)

        hsv[:,:,0] = 100
        hsv[:,:,1] = 2

        obs = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
        obs = cv2.cvtColor(obs, cv2.COLOR_RGB2GRAY)

        obs = np.expand_dims(obs, axis=-1)

        tree_obs = np.expand_dims(tree_obs,axis=-1)
        obs = np.concatenate((obs,tree_obs), axis=-1)

        return obs

class ActionShaping(gym.ActionWrapper):

    def __init__(self, env, camera_angle=15, always_attack=False):
        super().__init__(env)

        self.camera_angle = camera_angle
        self.always_attack = False
        self._actions = [
            [('attack', 1)],
            [('forward', 1)],
            # [('back', 1)],
            # [('left', 1)],
            # [('right', 1)],
            # [('jump', 1)],
            # [('forward', 1), ('attack', 1)],
            # [('craft', 'planks')],
            [('forward', 1), ('jump', 1)],
            [('camera', [-self.camera_angle, 0])],
            [('camera', [self.camera_angle, 0])],
            [('camera', [0, self.camera_angle])],
            [('camera', [0, -self.camera_angle])],
        ]

        self.actions = []
        for actions in self._actions:
            act = self.env.action_space.noop()
            for a, v in actions:
                act[a] = v
            if self.always_attack:
                act['attack'] = 1
            self.actions.append(act)

        self.action_space = gym.spaces.Discrete(len(self.actions))

    def action(self, action):
        return self.actions[action]

def track_exp(project_name=None):
    config = {
        "TRAIN_TIMESTEPS": 1000000,  # number of steps to train the agent for. At 70 FPS 2m steps take about 8 hours.
        "TRAIN_ENV": 'MineRLTreechop-v0',
        # training environment for the RL agent. Could use MineRLObtainDiamondDense-v0 here.
        "TRAIN_MODEL_NAME": 'potato',  # name to use when saving the trained agent.
        "TEST_MODEL_NAME": 'potato',  # name to use when loading the trained agent.
        "TEST_EPISODES": 10,  # number of episodes to test the agent for.
        "MAX_TEST_EPISODE_LEN": 18000,  # 18k is the default for MineRLObtainDiamond.
        "TREECHOP_STEPS": 2000,  # number of steps to run RL lumberjack for in evaluations.
        "RECORD_TRAINING_VIDEOS": False,  # if True, records videos of all episodes done during training.
        "RECORD_TEST_VIDEOS": False,  # if True, records videos of all episodes done during evaluation.
    }
    wandb.init(
        anonymous="allow",
        project=project_name,
        config=config,
        sync_tensorboard=True,
        name='v1',
        monitor_gym=True,
        save_code=True,
    )


def make_env(idx):
    def thunk():
        env = gym.make('MineRLObtainDiamond-v0')
        if idx == 0 and False:
            env = gym.wrappers.Monitor(env, f"train_videos/{'v1'}")
        env = PovOnlyObservation(env)
        env = ActionShaping(env, always_attack=True)
        env = gym.wrappers.RecordEpisodeStatistics(env)  # record stats such as returns
        return env
    return thunk

track_exp(project_name="minerl")

env = DummyVecEnv([make_env(i) for i in range(1)])
# For all the PPO hyperparameters you could tune see this:
# https://github.com/DLR-RM/stable-baselines3/blob/6f822b9ed7d6e8f57e5a58059923a5b24e8db283/stable_baselines3/ppo/ppo.py#L16
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=f"runs/{'v1'}")
model.learn(total_timesteps=2000000)  # 2m steps is about 8h at 70 FPS
model.save('ppo_first')
# env_checker.check_env(env, warn=True, skip_render_check=True)
# MineRL might throw an exception when closing on Windows, but it can be ignored (the environment does close).
try:
    env.close()
except Exception:
    pass

# env.actions.sample()
# obs.dtype
# np.ndarray.dtype(obs)
# env = gym.make('MineRLObtainDiamond-v0')
# env = PovOnlyObservation(env)
#env = ActionShaping(env, always_attack=True)
# # obs = env.reset()
#
# # Parameters:
# config = {
#     "TRAIN_TIMESTEPS": 2000000,  # number of steps to train the agent for. At 70 FPS 2m steps take about 8 hours.
#     "TRAIN_ENV": 'MineRLTreechop-v0',  # training environment for the RL agent. Could use MineRLObtainDiamondDense-v0 here.
#     "TRAIN_MODEL_NAME": 'potato',  # name to use when saving the trained agent.
#     "TEST_MODEL_NAME": 'potato',  # name to use when loading the trained agent.
#     "TEST_EPISODES": 10,  # number of episodes to test the agent for.
#     "MAX_TEST_EPISODE_LEN": 18000,  # 18k is the default for MineRLObtainDiamond.
#     "TREECHOP_STEPS": 2000,  # number of steps to run RL lumberjack for in evaluations.
#     "RECORD_TRAINING_VIDEOS": False,  # if True, records videos of all episodes done during training.
#     "RECORD_TEST_VIDEOS": False,  # if True, records videos of all episodes done during evaluation.
# }
# experiment_name = f"ppo_{int(time.time())}"
#
#
#
#
#
#
#
#
# def train():
#     env = DummyVecEnv([make_env(i) for i in range(1)])
#     # For all the PPO hyperparameters you could tune see this:
#     # https://github.com/DLR-RM/stable-baselines3/blob/6f822b9ed7d6e8f57e5a58059923a5b24e8db283/stable_baselines3/ppo/ppo.py#L16
#     model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=f"runs/{experiment_name}")
#     model.learn(total_timesteps=config["TRAIN_TIMESTEPS"])  # 2m steps is about 8h at 70 FPS
#     model.save(config["TRAIN_MODEL_NAME"])
#
#     # MineRL might throw an exception when closing on Windows, but it can be ignored (the environment does close).
#     try:
#         env.close()
#     except Exception:
#         pass
#
#
# def str_to_act(env, actions):
#     """
#     Simplifies specifying actions for the scripted part of the agent.
#     Some examples for a string with a single action:
#         'craft:planks'
#         'camera:[10,0]'
#         'attack'
#         'jump'
#         ''
#     There should be no spaces in single actions, as we use spaces to separate actions with multiple "buttons" pressed:
#         'attack sprint forward'
#         'forward camera:[0,10]'
#
#     :param env: base MineRL environment.
#     :param actions: string of actions.
#     :return: dict action, compatible with the base MineRL environment.
#     """
#     act = env.action_space.noop()
#     for action in actions.split():
#         if ":" in action:
#             k, v = action.split(':')
#             if k == 'camera':
#                 act[k] = eval(v)
#             else:
#                 act[k] = v
#         else:
#             act[action] = 1
#     return act
#
#
# def get_action_sequence():
#     """
#     Specify the action sequence for the scripted part of the agent.
#     """
#     # make planks, sticks, crafting table and wooden pickaxe:
#     action_sequence = []
#     action_sequence += [''] * 100
#     action_sequence += ['craft:planks'] * 4
#     action_sequence += ['craft:stick'] * 2
#     action_sequence += ['craft:crafting_table']
#     action_sequence += ['camera:[10,0]'] * 18
#     action_sequence += ['attack'] * 20
#     action_sequence += [''] * 10
#     action_sequence += ['jump']
#     action_sequence += [''] * 5
#     action_sequence += ['place:crafting_table']
#     action_sequence += [''] * 10
#
#     # bug: looking straight down at a crafting table doesn't let you craft. So we look up a bit before crafting.
#     action_sequence += ['camera:[-1,0]']
#     action_sequence += ['nearbyCraft:wooden_pickaxe']
#     action_sequence += ['camera:[1,0]']
#     action_sequence += [''] * 10
#     action_sequence += ['equip:wooden_pickaxe']
#     action_sequence += [''] * 10
#
#     # dig down:
#     action_sequence += ['attack'] * 600
#     action_sequence += [''] * 10
#
#     return action_sequence
#
#
# def test():
#     action_sequence = get_action_sequence()
#     writer = SummaryWriter(f"runs/{experiment_name}")
#     env = gym.make('MineRLObtainDiamond-v0').env
#     time_limit = min(config["MAX_TEST_EPISODE_LEN"], config["TREECHOP_STEPS"] + len(action_sequence))
#     env = gym.wrappers.TimeLimit(env, time_limit)
#
#     # optional interactive mode, where you can connect to your agent and play together (see link for details):
#     # https://minerl.io/docs/tutorials/minerl_tools.html#interactive-mode-minerl-interactor
#     # env.make_interactive(port=6666, realtime=True)
#
#     if config["RECORD_TEST_VIDEOS"]:
#         env = gym.wrappers.Monitor(env, f"test_videos/{experiment_name}")
#     env = PovOnlyObservation(env)
#     env = ActionShaping(env, always_attack=True)
#     env1 = env.env
#
#     model = PPO.load(config["TEST_MODEL_NAME"], verbose=1)
#     model.set_env(env)
#
#     for episode in range(config["TEST_EPISODES"]):
#         obs = env.reset()
#         done = False
#         total_reward = 0
#         steps = 0
#
#         # RL part to get some logs:
#         for i in range(config["TREECHOP_STEPS"]):
#             action = model.predict(obs.copy())
#             obs, reward, done, _ = env.step(action[0])
#             total_reward += reward
#             steps += 1
#             if done:
#                 break
#
#         # scripted part to use the logs:
#         if not done:
#             for i, action in enumerate(action_sequence[:config["MAX_TEST_EPISODE_LEN"] - config["TREECHOP_STEPS"]]):
#                 obs, reward, done, _ = env1.step(str_to_act(env1, action))
#                 total_reward += reward
#                 steps += 1
#                 if done:
#                     break
#
#         print(f'Episode #{episode + 1} return: {total_reward}\t\t episode length: {steps}')
#         writer.add_scalar("return", total_reward, global_step=episode)
#
#     env.close()
#
#
# def main():
#     # uncomment the following to upload the logs and videos to Weights and Biases
#     # track_exp(project_name="minerl")
#
#     # train()
#     test()
#
#
# if __name__ == '__main__':
#     main()


env = gym.make('MineRLObtainDiamond-v0')
env = PovOnlyObservation(env)
env = ActionShaping(env, always_attack=True)
env1 = env.env
cv2.namedWindow('image', cv2.WINDOW_KEEPRATIO)
cv2.resizeWindow('image', 1000, 1000)

for episode in range(3):
    obs = env.reset()
    done = False
    total_reward = 0
    steps = 0

    # RL part to get some logs:
    for i in range(10000):
        #action = model.predict(obs.copy())
        obs, reward, done, _ = env.step(4)
        total_reward += reward
        env.render()
        steps += 1
        cv2.imshow('image', obs[:,:,0])
        cv2.resizeWindow('image', 950, 950)
        if done:
            break
        if cv2.waitKey(10) & 0xFF == ord('o'):
            break
    # scripted part to use the logs:
    # if not done:
    #     for i, action in enumerate(action_sequence[:config["MAX_TEST_EPISODE_LEN"] - config["TREECHOP_STEPS"]]):
    #         obs, reward, done, _ = env1.step(str_to_act(env1, action))
    #         total_reward += reward
    #         steps += 1
    #         if done:
    #             break
    #
    # print(f'Episode #{episode + 1} return: {total_reward}\t\t episode length: {steps}')
    # writer.add_scalar("return", total_reward, global_step=episode)

env.close()